# 2.6. Probability and Statistics
---

## Introduction

Probability theory provides the mathematical foundation for reasoning under uncertainty. In machine learning and artificial intelligence, we rarely have complete information about the world. Instead, we must make decisions and predictions based on incomplete or noisy data. Probability theory gives us the tools to quantify uncertainty, make principled inferences, and learn from data.

Statistical methods extend probability theory to analyze real-world data, estimate unknown parameters, test hypotheses, and make predictions. Together, probability and statistics form the backbone of modern data science, enabling everything from spam filters to medical diagnosis systems to autonomous vehicles.

This chapter introduces the fundamental concepts of probability theory and statistical inference, with particular emphasis on applications in computer science and machine learning. We begin with the axiomatic foundations of probability, develop the key concepts of random variables and distributions, and explore how these tools enable us to reason about uncertainty in computational systems.

**Key Questions Addressed:**
- How do we formally model uncertainty?
- How can we update beliefs when we observe new evidence?
- How do we make optimal decisions under uncertainty?
- How do we learn from data when the underlying process is random?

---

## Problem Statement

**Motivating Problem: Spam Email Classification**

Consider the problem of automatically classifying emails as spam or legitimate. This problem exhibits all the key challenges that probability theory addresses:

**The Challenge:**
- **Uncertainty:** We cannot be certain whether an email is spam based solely on its content
- **Partial Information:** We only observe the email text, not the sender's true intent
- **Variability:** Different spam emails use different words and patterns
- **Learning Required:** The system must learn what characterizes spam from examples

**Probabilistic Approach:**

Instead of deterministic rules (e.g., "if email contains 'FREE MONEY' then spam"), we model:

$$P(\text{Spam} \mid \text{Email Content})$$

This probability represents our degree of belief that the email is spam given the observed content.

**Using Bayes' Theorem:**
$$P(\text{Spam} \mid \text{Words}) = \frac{P(\text{Words} \mid \text{Spam}) \cdot P(\text{Spam})}{P(\text{Words})}$$

Where:
- $P(\text{Spam})$: Prior probability (base rate of spam)
- $P(\text{Words} \mid \text{Spam})$: Likelihood (how likely these words appear in spam)
- $P(\text{Spam} \mid \text{Words})$: Posterior probability (our belief after seeing the email)

**Why This Matters:**

This probabilistic framework allows us to:
1. Quantify uncertainty rather than making hard decisions
2. Update beliefs as we observe new evidence
3. Make optimal decisions that account for different types of errors
4. Learn from data systematically

The same principles apply to medical diagnosis, autonomous driving, speech recognition, and countless other applications where uncertainty is inherent.

---

## Applications in Computer Science

Probability and statistics are fundamental to numerous areas of computer science and machine learning:

### 1. Machine Learning

**Supervised Learning:**
- **Classification:** Assigning probabilities to class labels (spam detection, image recognition)
- **Regression:** Modeling uncertainty in predictions (stock prices, weather forecasting)
- **Bayesian Networks:** Representing dependencies between random variables

**Unsupervised Learning:**
- **Clustering:** Modeling data as mixtures of probability distributions
- **Dimensionality Reduction:** Finding probabilistic latent representations
- **Generative Models:** Learning probability distributions that generate data

### 2. Natural Language Processing

**Statistical Language Models:**
- Computing probabilities of word sequences: $P(\text{word}_1, \text{word}_2, \ldots, \text{word}_n)$
- Machine translation using conditional probabilities
- Speech recognition combining acoustic and language models

**Applications:**
- Autocomplete and text prediction
- Sentiment analysis
- Named entity recognition

### 3. Computer Vision

**Image Classification:**
- Probabilistic models for object recognition
- Bayesian approaches to handling occlusion and noise
- Uncertainty quantification in autonomous driving

**Image Generation:**
- Generative Adversarial Networks (GANs)
- Variational Autoencoders (VAEs)
- Diffusion models

### 4. Information Retrieval and Search

**Search Engines:**
- Ranking documents by relevance probability: $P(\text{Relevant} \mid \text{Query}, \text{Document})$
- Click-through rate prediction
- Personalized recommendations

**Recommendation Systems:**
- Collaborative filtering using probabilistic matrix factorization
- Modeling user preferences as probability distributions
- A/B testing for algorithm improvements

### 5. Cybersecurity

**Intrusion Detection:**
- Anomaly detection using statistical models
- Identifying unusual network traffic patterns
- Bayesian approaches to threat assessment

**Cryptography:**
- Randomness testing for key generation
- Statistical attacks on encryption schemes

### 6. Distributed Systems and Networks

**Performance Modeling:**
- Queueing theory for analyzing system performance
- Reliability analysis using probability distributions
- Load balancing under uncertain demand

**Network Protocols:**
- Random backoff in collision detection (Ethernet)
- Probabilistic routing algorithms
- Gossip protocols for information dissemination

### 7. Algorithmic Design

**Randomized Algorithms:**
- QuickSort with random pivot selection
- Monte Carlo methods for numerical integration
- Las Vegas algorithms with guaranteed correctness

**Probabilistic Data Structures:**
- Bloom filters for membership testing
- Count-Min Sketch for frequency estimation
- HyperLogLog for cardinality estimation

### 8. Reinforcement Learning

**Decision Making Under Uncertainty:**
- Modeling environments as Markov Decision Processes (MDPs)
- Policy evaluation using expected returns
- Exploration-exploitation tradeoffs

**Applications:**
- Game playing (AlphaGo, Chess engines)
- Robotics control
- Resource allocation

### 9. Bioinformatics

**Sequence Analysis:**
- Hidden Markov Models for gene finding
- Probabilistic alignment algorithms
- Statistical significance testing

### 10. A/B Testing and Experimentation

**Product Development:**
- Statistical hypothesis testing for feature comparison
- Bayesian optimization for hyperparameter tuning
- Confidence intervals for performance metrics

---

**Common Thread:**

All these applications share the need to:
1. **Model Uncertainty:** Represent what we don't know
2. **Learn from Data:** Update models based on observations
3. **Make Decisions:** Choose actions that maximize expected utility
4. **Quantify Confidence:** Provide measures of reliability

The mathematical tools developed in this chapter—probability axioms, conditional probability, Bayes' theorem, random variables, expectation, and variance—provide the foundation for all these applications.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib for better-looking plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['lines.linewidth'] = 2

print("Probability and Statistics - D2L Chapter 2.6")
print("="*70)
print("Environment initialized successfully")
print("Libraries: numpy, matplotlib, scipy.stats")

---

## Concrete Example: Building a Spam Filter

To illustrate the practical application of probability theory, we develop a simple spam filter using the principles introduced in this chapter.

### Problem Setup

**Data:**
- Training set: 1000 emails (700 legitimate, 300 spam)
- Each email is represented by the presence/absence of specific words

**Approach:**
- Use **Naive Bayes classification**
- Compute $P(\text{Spam} \mid \text{Word}_1, \text{Word}_2, \ldots)$
- Make decision based on posterior probability

### Assumptions

**Naive Bayes Assumption:**
Given the class (spam or not), word occurrences are independent:
$$P(\text{Word}_1, \ldots, \text{Word}_n \mid \text{Spam}) = \prod_{i=1}^n P(\text{Word}_i \mid \text{Spam})$$

While this assumption is often violated in practice, Naive Bayes classifiers work remarkably well for text classification.

### Classification Rule

For a new email with words $\{w_1, w_2, \ldots, w_n\}$:

$$P(\text{Spam} \mid w_1, \ldots, w_n) = \frac{P(w_1, \ldots, w_n \mid \text{Spam}) P(\text{Spam})}{P(w_1, \ldots, w_n)}$$

Using the Naive Bayes assumption:
$$P(\text{Spam} \mid w_1, \ldots, w_n) \propto P(\text{Spam}) \prod_{i=1}^n P(w_i \mid \text{Spam})$$

**Decision:** Classify as spam if $P(\text{Spam} \mid \text{words}) > 0.5$

In [ ]:
# Concrete Example: Spam Filter Implementation

# Simulate training data
np.random.seed(42)

# Word probabilities (simplified model)
# P(word | spam) and P(word | legitimate)
words = ['free', 'money', 'winner', 'meeting', 'report', 'project']

# Probability of each word appearing in spam vs legitimate emails
word_probs = {
    'free':    {'spam': 0.60, 'legit': 0.05},
    'money':   {'spam': 0.50, 'legit': 0.08},
    'winner':  {'spam': 0.45, 'legit': 0.02},
    'meeting': {'spam': 0.10, 'legit': 0.40},
    'report':  {'spam': 0.08, 'legit': 0.35},
    'project': {'spam': 0.05, 'legit': 0.45}
}

# Prior probabilities
p_spam = 0.30  # 30% of emails are spam
p_legit = 0.70

# Test emails
test_emails = [
    {'content': ['free', 'money', 'winner'], 'true_label': 'spam'},
    {'content': ['meeting', 'project', 'report'], 'true_label': 'legit'},
    {'content': ['free', 'project'], 'true_label': 'uncertain'},
    {'content': ['money'], 'true_label': 'spam'}
]

print("="*70)
print("SPAM FILTER DEMONSTRATION USING NAIVE BAYES")
print("="*70)

print("\nTraining Data:")
print(f"  Prior P(Spam) = {p_spam:.2f}")
print(f"  Prior P(Legitimate) = {p_legit:.2f}")

print("\nWord Probabilities:")
print(f"{'Word':<12} {'P(word|Spam)':<15} {'P(word|Legit)'}")
print("-"*70)
for word in words:
    print(f"{word:<12} {word_probs[word]['spam']:<15.2f} {word_probs[word]['legit']:.2f}")

print("\n" + "="*70)
print("CLASSIFICATION RESULTS")
print("="*70)

results = []

for idx, email in enumerate(test_emails, 1):
    print(f"\nEmail {idx}: {email['content']}")
    print("-"*70)
    
    # Compute P(words | spam) * P(spam)
    prob_spam = p_spam
    for word in email['content']:
        prob_spam *= word_probs[word]['spam']
    
    # Compute P(words | legit) * P(legit)
    prob_legit = p_legit
    for word in email['content']:
        prob_legit *= word_probs[word]['legit']
    
    # Normalize to get posterior
    total = prob_spam + prob_legit
    posterior_spam = prob_spam / total
    posterior_legit = prob_legit / total
    
    # Decision
    prediction = 'spam' if posterior_spam > 0.5 else 'legit'
    
    print(f"  P(words | spam) × P(spam) = {prob_spam:.6f}")
    print(f"  P(words | legit) × P(legit) = {prob_legit:.6f}")
    print(f"  P(spam | words) = {posterior_spam:.4f} = {posterior_spam*100:.2f}%")
    print(f"  P(legit | words) = {posterior_legit:.4f} = {posterior_legit*100:.2f}%")
    print(f"  → Prediction: {prediction.upper()}")
    
    results.append({
        'words': email['content'],
        'p_spam': posterior_spam,
        'prediction': prediction,
        'true_label': email['true_label']
    })

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Posterior probabilities for each email
email_labels = [f"Email {i+1}\n{' '.join(r['words'][:2])}" for i, r in enumerate(results)]
spam_probs = [r['p_spam'] for r in results]
legit_probs = [1 - r['p_spam'] for r in results]

x = np.arange(len(email_labels))
width = 0.35

bars1 = ax1.bar(x - width/2, spam_probs, width, label='P(Spam|words)', color='red', alpha=0.7)
bars2 = ax1.bar(x + width/2, legit_probs, width, label='P(Legit|words)', color='green', alpha=0.7)

ax1.set_ylabel('Posterior Probability')
ax1.set_title('Naive Bayes Classification Results')
ax1.set_xticks(x)
ax1.set_xticklabels(email_labels, fontsize=8)
ax1.legend()
ax1.axhline(y=0.5, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax1.grid(True, alpha=0.3, axis='y')

# Add percentage labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height*100:.1f}%', ha='center', va='bottom', fontsize=8)

# Plot 2: Word likelihood ratios
likelihood_ratios = {}
for word in words:
    ratio = word_probs[word]['spam'] / word_probs[word]['legit']
    likelihood_ratios[word] = ratio

words_sorted = sorted(words, key=lambda w: likelihood_ratios[w], reverse=True)
ratios_sorted = [likelihood_ratios[w] for w in words_sorted]
colors = ['red' if r > 1 else 'green' for r in ratios_sorted]

ax2.barh(words_sorted, ratios_sorted, color=colors, alpha=0.7)
ax2.set_xlabel('Likelihood Ratio: P(word|Spam) / P(word|Legit)')
ax2.set_title('Spam Indicators: Likelihood Ratios')
ax2.axvline(x=1, color='black', linestyle='--', linewidth=2, label='Neutral (ratio=1)')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='x')

# Add ratio labels
for i, (word, ratio) in enumerate(zip(words_sorted, ratios_sorted)):
    ax2.text(ratio + 0.2, i, f'{ratio:.1f}x', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("KEY INSIGHTS:")
print("="*70)
print("1. Words like 'free', 'money', 'winner' are strong spam indicators")
print("2. Words like 'meeting', 'project', 'report' suggest legitimate emails")
print("3. Bayes' theorem allows us to combine multiple pieces of evidence")
print("4. The prior probability P(spam) affects final classification")
print("5. This probabilistic approach quantifies uncertainty in predictions")
print("="*70)

## Section 1: Core Definitions and Axioms of Probability

### 1.1 Sample Space and Events

**Definition 1.1 (Sample Space):**
The **sample space**, denoted $\mathcal{S}$ (or sometimes $\Omega$), is the set of all possible outcomes of a random experiment.

**Definition 1.2 (Event):**
An **event** $\mathcal{A}$ is a subset of the sample space: $\mathcal{A} \subseteq \mathcal{S}$.

**Definition 1.3 (Event Occurrence):**
We say that event $\mathcal{A}$ has occurred if and only if the realized outcome $z$ satisfies $z \in \mathcal{A}$.

**Examples:**
- Coin flip: $\mathcal{S} = \{\text{Heads}, \text{Tails}\}$
- Die roll: $\mathcal{S} = \{1, 2, 3, 4, 5, 6\}$
- Temperature measurement: $\mathcal{S} = \mathbb{R}$

**Reviewer's Commentary:**
- These definitions are standard and correct.
- The notation $\mathcal{S}$ for sample space is conventional (alternative notations include $\Omega$ or $S$).
- Events form a $\sigma$-algebra (discussed in Section 12).

### 1.2 Kolmogorov Axioms of Probability

**Definition 1.4 (Probability Function):**
A **probability function** (or probability measure) is a function
$$P: \mathcal{F} \to [0,1]$$
where $\mathcal{F}$ is a $\sigma$-algebra of events on $\mathcal{S}$, that satisfies the following three axioms:

**Axiom 1 (Non-negativity):** For any event $\mathcal{A} \in \mathcal{F}$,
$$P(\mathcal{A}) \geq 0$$

**Axiom 2 (Normalization):** The probability of the entire sample space is 1:
$$P(\mathcal{S}) = 1$$

**Axiom 3 (Countable Additivity):** For any countable collection of **mutually exclusive** (disjoint) events $\{\mathcal{A}_i\}_{i=1}^{\infty}$ where $\mathcal{A}_i \cap \mathcal{A}_j = \emptyset$ for all $i \neq j$,
$$P\left(\bigcup_{i=1}^{\infty} \mathcal{A}_i\right) = \sum_{i=1}^{\infty} P(\mathcal{A}_i)$$

**Reviewer's Commentary:**
- These are the **Kolmogorov axioms**, the foundation of modern probability theory (1933).
- **Critical requirement:** Axiom 3 requires that events be **mutually exclusive** (disjoint). This condition must be stated explicitly.
- For finite cases, we often use the simpler finite additivity: $P(A \cup B) = P(A) + P(B)$ when $A \cap B = \emptyset$.

### 1.3 Immediate Consequences of the Axioms

**Theorem 1.1 (Probability of Empty Set):**
$$P(\emptyset) = 0$$

**Proof:**

**Step 1:** Consider the sample space $\mathcal{S}$ and the empty set $\emptyset$.

**Step 2:** Note that $\mathcal{S} \cap \emptyset = \emptyset$ and $\mathcal{S} \cup \emptyset = \mathcal{S}$.

*This follows from the definition of the empty set.*

**Step 3:** Therefore $\mathcal{S}$ and $\emptyset$ are disjoint events.

*This follows directly from Step 2.*

**Step 4:** By Axiom 3 (countable additivity with $n=2$):
$$P(\mathcal{S} \cup \emptyset) = P(\mathcal{S}) + P(\emptyset)$$

*The additivity axiom applies since the events are disjoint.*

**Step 5:** Simplify the left side using $\mathcal{S} \cup \emptyset = \mathcal{S}$:
$$P(\mathcal{S}) = P(\mathcal{S}) + P(\emptyset)$$

**Step 6:** Subtract $P(\mathcal{S})$ from both sides:
$$0 = P(\emptyset)$$

Therefore $P(\emptyset) = 0$. 

**Theorem 1.2 (Complement Rule):**
For any event $\mathcal{A}$ and its complement $\mathcal{A}^c = \mathcal{S} \setminus \mathcal{A}$,
$$P(\mathcal{A}) + P(\mathcal{A}^c) = 1$$

**Proof:**

**Step 1:** By definition of complement, $\mathcal{A} \cap \mathcal{A}^c = \emptyset$.

*An event and its complement are disjoint by definition.*

**Step 2:** Also by definition of complement, $\mathcal{A} \cup \mathcal{A}^c = \mathcal{S}$.

*An event and its complement together partition the sample space.*

**Step 3:** Since $\mathcal{A}$ and $\mathcal{A}^c$ are disjoint, by Axiom 3:
$$P(\mathcal{A} \cup \mathcal{A}^c) = P(\mathcal{A}) + P(\mathcal{A}^c)$$

**Step 4:** Substitute from Step 2:
$$P(\mathcal{S}) = P(\mathcal{A}) + P(\mathcal{A}^c)$$

**Step 5:** By Axiom 2, $P(\mathcal{S}) = 1$, therefore:
$$1 = P(\mathcal{A}) + P(\mathcal{A}^c)$$

Therefore $P(\mathcal{A}) + P(\mathcal{A}^c) = 1$. 

**Corollary 1.2.1:**
$$P(\mathcal{A}^c) = 1 - P(\mathcal{A})$$

**Theorem 1.3 (Monotonicity):**
If $\mathcal{A} \subseteq \mathcal{B}$, then $P(\mathcal{A}) \leq P(\mathcal{B})$.

**Proof:**

**Step 1:** Write $\mathcal{B} = \mathcal{A} \cup (\mathcal{B} \setminus \mathcal{A})$.

*Any set can be decomposed into a subset and its relative complement.*

**Step 2:** Note that $\mathcal{A}$ and $\mathcal{B} \setminus \mathcal{A}$ are disjoint.

**Step 3:** By Axiom 3:
$$P(\mathcal{B}) = P(\mathcal{A}) + P(\mathcal{B} \setminus \mathcal{A})$$

**Step 4:** By Axiom 1, $P(\mathcal{B} \setminus \mathcal{A}) \geq 0$.

**Step 5:** Therefore:
$$P(\mathcal{B}) = P(\mathcal{A}) + P(\mathcal{B} \setminus \mathcal{A}) \geq P(\mathcal{A})$$



---

## Section 2: Random Variables

### 2.1 What is a Random Variable?

A random variable provides a mathematical framework for mapping outcomes from a sample space to numerical values, enabling quantitative analysis.

**Definition 2.1 (Random Variable - Formal Version):**
A **random variable** is a measurable function $X: \mathcal{S} \to \mathbb{R}$ that maps outcomes from the sample space to real numbers.

**Intuition:** A random variable assigns a number to each possible outcome of a random experiment.

**Example 1: Coin Toss**

Consider a coin flip experiment where the sample space is $\mathcal{S} = \{\text{Heads}, \text{Tails}\}$.

Define a random variable $X$ that maps:
- Heads → 1
- Tails → 0

This encoding enables mathematical operations such as computing averages.

**Example 2: Rolling a Die**

When rolling a die, define $X$ to be the number showing on the top face.

The random variable takes values in $\{1, 2, 3, 4, 5, 6\}$.

**Two Types of Random Variables:**

| Type | Description | Examples |
|------|-------------|----------|
| **Discrete** | Countable values | Coin flips, dice rolls, number of customers |
| **Continuous** | Any value in a range | Height, weight, temperature, time |

### 2.2 Probability Mass Function (PMF) and Probability Density Function (PDF)

**Definition 2.2 (Probability Mass Function):**
For a discrete random variable $X$, the **probability mass function (PMF)** is:
$$p_X(x) = P(X = x)$$

**Properties of PMF:**
1. $p_X(x) \geq 0$ for all $x$
2. $\sum_{x} p_X(x) = 1$

**Definition 2.3 (Probability Density Function):**
For a continuous random variable $X$, the **probability density function (PDF)** is a function $f_X(x)$ such that:
$$P(a \leq X \leq b) = \int_a^b f_X(x) \, dx$$

**Properties of PDF:**
1. $f_X(x) \geq 0$ for all $x$
2. $\int_{-\infty}^{\infty} f_X(x) \, dx = 1$

**Important Note:** For continuous random variables, $P(X = x) = 0$ for any specific value $x$. We can only compute probabilities for intervals.

### 2.3 Cumulative Distribution Function (CDF)

**Definition 2.4 (Cumulative Distribution Function):**
For any random variable $X$, the **cumulative distribution function (CDF)** is:
$$F_X(x) = P(X \leq x)$$

**Properties of CDF:**
1. $\lim_{x \to -\infty} F_X(x) = 0$
2. $\lim_{x \to +\infty} F_X(x) = 1$
3. $F_X$ is non-decreasing: if $a < b$, then $F_X(a) \leq F_X(b)$
4. $F_X$ is right-continuous

**Relationship between PDF and CDF (continuous case):**
$$f_X(x) = \frac{d}{dx}F_X(x)$$
$$F_X(x) = \int_{-\infty}^{x} f_X(t) \, dt$$

**Computing Probabilities:**
$$P(a < X \leq b) = F_X(b) - F_X(a)$$

### 2.4 Example: The Fair Coin Toss

Consider a fair coin flip experiment.

**Setup:**
- Fair coin with equal probability of heads or tails
- Sample space: $\mathcal{S} = \{\text{Heads}, \text{Tails}\}$

**Step 1: Define a Random Variable**

Define $X = 1$ if Heads, and $X = 0$ if Tails.

Thus:
- $P(X = 1) = 0.5$ (probability of heads)
- $P(X = 0) = 0.5$ (probability of tails)

**Step 2: Calculate the Expected Value**

The expected value is computed as:

$$E[X] = \sum_{x} x \cdot P(X = x) = (1)(0.5) + (0)(0.5) = 0.5$$

**Interpretation:** As the number of flips approaches infinity, the sample average converges to 0.5.

**Step 3: Calculate the Variance**

Variance measures the dispersion of the distribution:

Using the formula: $\text{Var}[X] = E[X^2] - (E[X])^2$

First, compute $E[X^2]$:
$$E[X^2] = (1^2)(0.5) + (0^2)(0.5) = 0.5$$

Then:
$$\text{Var}[X] = 0.5 - (0.5)^2 = 0.5 - 0.25 = 0.25$$

Standard deviation:
$$\sigma = \sqrt{0.25} = 0.5$$

**Interpretation:** The dispersion is maximal for a Bernoulli random variable, as the outcomes are binary with equal probability.

### 2.5 Example: Two Coin Tosses

**Sample space:** $\mathcal{S} = \{HH, HT, TH, TT\}$

Define $Y$ to be the total number of heads. The random variable takes values in $\{0, 1, 2\}$.

**Probabilities:**
- $P(Y = 0) = 1/4$ (both tails: TT)
- $P(Y = 1) = 2/4 = 1/2$ (one head: HT or TH)
- $P(Y = 2) = 1/4$ (both heads: HH)

**Expected Value:**
$$E[Y] = (0)(1/4) + (1)(1/2) + (2)(1/4) = 0 + 0.5 + 0.5 = 1$$

The expected number of heads is 1.

**Variance:**
$$E[Y^2] = (0^2)(1/4) + (1^2)(1/2) + (2^2)(1/4) = 0 + 0.5 + 1 = 1.5$$

$$\text{Var}[Y] = 1.5 - (1)^2 = 1.5 - 1 = 0.5$$

**Observation:** For 1 coin, $E[X] = 0.5$ and for 2 coins, $E[Y] = 1 = 2 \times 0.5$. This demonstrates the **linearity of expectation** property.

### 2.6 Additional Examples

#### Example 1: Weather Forecasting

When a forecast indicates "30% chance of rain tomorrow," this denotes a probability assignment.

**Sample Space:** $\mathcal{S} = \{\text{Rain}, \text{No Rain}\}$

**Probabilities:**
- $P(\text{Rain}) = 0.3$
- $P(\text{No Rain}) = 0.7$

Define $X = 1$ if it rains, $X = 0$ otherwise.

**Expected value:**
$$E[X] = (1)(0.3) + (0)(0.7) = 0.3$$

**Interpretation:** Under frequentist interpretation, if there were 100 days with identical atmospheric conditions, rain would occur on approximately 30 of them.

---

#### Example 2: Exam Scores

Suppose exam performance follows this distribution:
- Score 90-100: probability 0.4
- Score 70-89: probability 0.5
- Score 60-69: probability 0.1

Using the midpoint of each range:

$$E[\text{Score}] = (95)(0.4) + (80)(0.5) + (65)(0.1) = 38 + 40 + 6.5 = 84.5$$

Expected score: 84.5

**Variance calculation:**
$$E[\text{Score}^2] = (95^2)(0.4) + (80^2)(0.5) + (65^2)(0.1) = 3610 + 3200 + 422.5 = 7232.5$$

$$\text{Var}[\text{Score}] = 7232.5 - (84.5)^2 = 7232.5 - 7140.25 = 92.25$$

$$\sigma = \sqrt{92.25} \approx 9.6$$

**Interpretation:** The standard deviation of approximately 10 points indicates the typical deviation from the expected score.

---

#### Example 3: Probabilistic Reward System

A reward system has:
- Common item (value 1): 70% probability
- Rare item (value 5): 25% probability
- Legendary item (value 50): 5% probability

**Expected value:**
$$E[X] = (1)(0.70) + (5)(0.25) + (50)(0.05) = 0.70 + 1.25 + 2.50 = 4.45$$

Each trial has an expected value of 4.45 units.

**Variance:**
$$E[X^2] = (1^2)(0.70) + (5^2)(0.25) + (50^2)(0.05) = 0.70 + 6.25 + 125 = 131.95$$

$$\text{Var}[X] = 131.95 - (4.45)^2 = 131.95 - 19.80 = 112.15$$

$$\sigma = \sqrt{112.15} \approx 10.59$$

**Analysis:** The high variance (112.15) relative to the mean (4.45) indicates significant variability in outcomes. Most trials yield low values, while rare high-value outcomes substantially influence the expected value.

---

## Section 3: Joint and Conditional Probability

### 3.1 Joint Probability

**Definition 3.1 (Joint Probability):**
For two random variables $A$ and $B$, the **joint probability** is:
$$P(A = a, B = b)$$
which represents the probability that both events $\{A = a\}$ and $\{B = b\}$ occur simultaneously.

**Alternative notation:** $P(A = a \cap B = b)$ or $P(A = a \text{ and } B = b)$

**Theorem 3.1 (Joint Probability Bounds):**
For any values $a, b$:
$$P(A = a, B = b) \leq P(A = a)$$
$$P(A = a, B = b) \leq P(B = b)$$

**Proof of first inequality:**

**Step 1:** Partition the event $\{A = a\}$ based on all possible values of $B$:
$$\{A = a\} = \bigcup_{v \in \text{Val}(B)} \{A = a, B = v\}$$

*This follows from the law of total probability.*

**Step 2:** The events $\{A = a, B = v\}$ for different values of $v$ are mutually exclusive.

*$B$ cannot take two different values simultaneously.*

**Step 3:** Apply Axiom 3 (countable additivity):
$$P(A = a) = \sum_{v \in \text{Val}(B)} P(A = a, B = v)$$

**Step 4:** Since all probabilities are non-negative (Axiom 1), each term in the sum is $\geq 0$:
$$P(A = a) = P(A = a, B = b) + \sum_{v \neq b} P(A = a, B = v) \geq P(A = a, B = b)$$

Therefore $P(A = a, B = b) \leq P(A = a)$. The proof for the second inequality is symmetric. 

### 3.2 Conditional Probability

**Definition 3.2 (Conditional Probability):**
The **conditional probability** of event $B = b$ given that event $A = a$ has occurred is defined as:
$$P(B = b \mid A = a) = \frac{P(A = a, B = b)}{P(A = a)}$$
provided that $P(A = a) > 0$.

**Interpretation:**
- We restrict our sample space to outcomes where $A = a$ occurred
- We renormalize probabilities so they sum to 1 over this restricted space
- The conditional probability measures the likelihood of $B = b$ within this restricted space

**Important Condition:** This definition is only valid when $P(A = a) > 0$. If $P(A = a) = 0$, conditional probability is undefined (division by zero).

**Example:**

Roll a fair die. Let:
- $A$ = "roll is even" = $\{2, 4, 6\}$
- $B$ = "roll is greater than 3" = $\{4, 5, 6\}$

Then:
- $P(A) = 3/6 = 1/2$
- $P(B) = 3/6 = 1/2$
- $P(A \cap B) = P(\{4, 6\}) = 2/6 = 1/3$

$$P(B \mid A) = \frac{P(A \cap B)}{P(A)} = \frac{1/3}{1/2} = \frac{2}{3}$$

Given that the roll is even, there's a 2/3 chance it's greater than 3.

### 3.3 Verification that Conditional Probability is a Valid Probability

**Theorem 3.2:** For fixed $A = a$ with $P(A = a) > 0$, the function $Q(B = b) = P(B = b \mid A = a)$ satisfies all probability axioms.

**Proof:**

**Axiom 1 (Non-negativity):**

**Step 1:** By definition:
$$Q(B = b) = P(B = b \mid A = a) = \frac{P(A = a, B = b)}{P(A = a)}$$

**Step 2:** The numerator satisfies $P(A = a, B = b) \geq 0$ (by Axiom 1).

**Step 3:** The denominator satisfies $P(A = a) > 0$ (given assumption).

**Step 4:** Therefore:
$$Q(B = b) = \frac{P(A = a, B = b)}{P(A = a)} \geq 0$$

**Axiom 2 (Normalization):**

**Step 1:** Sum over all possible values of $B$:
$$\sum_{b} Q(B = b) = \sum_{b} \frac{P(A = a, B = b)}{P(A = a)}$$

**Step 2:** Factor out constant denominator:
$$= \frac{1}{P(A = a)} \sum_{b} P(A = a, B = b)$$

**Step 3:** By marginalization:
$$= \frac{1}{P(A = a)} \cdot P(A = a) = 1$$

**Axiom 3 (Additivity):** Similar verification for disjoint events.

Therefore, conditional probability defines a valid probability measure.

### 3.4 Marginalization (Law of Total Probability)

**Theorem 3.3 (Marginalization):**
For random variables $A$ and $B$:
$$P(A = a) = \sum_{v \in \text{Val}(B)} P(A = a, B = v)$$

**Proof:** This was proven in Theorem 3.1.

**Theorem 3.4 (Law of Total Probability - Alternative Form):**
$$P(A = a) = \sum_{v \in \text{Val}(B)} P(A = a \mid B = v) P(B = v)$$

**Proof:**

**Step 1:** Start with the marginalization formula:
$$P(A = a) = \sum_{v} P(A = a, B = v)$$

**Step 2:** Apply the definition of conditional probability to each term:
$$P(A = a, B = v) = P(A = a \mid B = v) P(B = v)$$

*This assumes $P(B = v) > 0$.*

**Step 3:** Substitute into Step 1:
$$P(A = a) = \sum_{v} P(A = a \mid B = v) P(B = v)$$

**Note:** For values $v$ where $P(B = v) = 0$, the term contributes 0 to the sum. 

**Intuition:** To find the probability of $A$, sum over all possible "paths" through values of $B$.

---

## Section 4: Bayes' Theorem

### 4.1 Derivation of Bayes' Theorem

**Theorem 4.1 (Bayes' Theorem - Basic Form):**
For events $A$ and $B$ with $P(A) > 0$ and $P(B) > 0$:
$$P(A \mid B) = \frac{P(B \mid A) P(A)}{P(B)}$$

**Proof:**

**Step 1:** By definition of conditional probability:
$$P(A \mid B) = \frac{P(A, B)}{P(B)}$$

*This is valid when $P(B) > 0$.*

**Step 2:** Note that joint probability is symmetric:
$$P(A, B) = P(B, A)$$

*$P(A \cap B) = P(B \cap A)$.*

**Step 3:** Apply definition of conditional probability to express joint probability:
$$P(B, A) = P(B \mid A) P(A)$$

*This is valid when $P(A) > 0$.*

**Step 4:** Combine Steps 2 and 3:
$$P(A, B) = P(B \mid A) P(A)$$

**Step 5:** Substitute Step 4 into Step 1:
$$P(A \mid B) = \frac{P(B \mid A) P(A)}{P(B)}$$

Therefore Bayes' theorem is established.

### 4.2 Terminology and Interpretation

In Bayes' theorem:
$$P(A \mid B) = \frac{P(B \mid A) \cdot P(A)}{P(B)}$$

| Term | Name | Meaning |
|------|------|--------|
| $P(A)$ | **Prior** | What we believed BEFORE seeing evidence |
| $P(B \mid A)$ | **Likelihood** | How likely is the evidence if our belief is true? |
| $P(B)$ | **Marginal / Evidence** | How likely is the evidence overall? |
| $P(A \mid B)$ | **Posterior** | What we believe AFTER seeing evidence |

**The Big Idea:** Bayes' theorem tells us how to update our beliefs when we get new information.

### 4.3 Bayes' Theorem with Marginalization

**Theorem 4.2 (Bayes' Theorem - Normalized Form):**
When $P(B)$ is unknown, we can compute it via marginalization:
$$P(A \mid B) = \frac{P(B \mid A) P(A)}{\sum_{a'} P(B \mid A = a') P(A = a')}$$

**Proof:**

**Step 1:** Start with Bayes' theorem:
$$P(A \mid B) = \frac{P(B \mid A) P(A)}{P(B)}$$

**Step 2:** Apply marginalization to compute $P(B)$:
$$P(B) = \sum_{a'} P(B, A = a') = \sum_{a'} P(B \mid A = a') P(A = a')$$

**Step 3:** Substitute into Step 1:
$$P(A \mid B) = \frac{P(B \mid A) P(A)}{\sum_{a'} P(B \mid A = a') P(A = a')}$$



### 4.4 Proportional Form of Bayes' Theorem

**Theorem 4.3 (Bayes' Theorem - Proportional Form):**
$$P(A \mid B) \propto P(B \mid A) P(A)$$

**Explanation:** The proportionality symbol $\propto$ means "proportional to" or "equal up to a normalization constant."

**Detailed Meaning:**

**Step 1:** From Bayes' theorem:
$$P(A \mid B) = \frac{P(B \mid A) P(A)}{P(B)}$$

**Step 2:** When computing $P(A \mid B)$ for different values of $A$ with $B$ fixed, the denominator $P(B)$ is constant.

**Step 3:** Therefore:
$$P(A \mid B) = \frac{1}{P(B)} \cdot P(B \mid A) P(A) \propto P(B \mid A) P(A)$$

**Step 4:** To recover the full probability, we normalize:
$$P(A = a \mid B) = \frac{P(B \mid A = a) P(A = a)}{\sum_{a'} P(B \mid A = a') P(A = a')}$$

**Usage:** This form is useful when we only need to compare relative probabilities or when we'll normalize at the end.

### 4.5 Example: Friend Attendance Prediction

**Situation:** Consider a friend who states they will attend a party. Historical data shows:
- When they actually attend, they always confirm attendance: $P(\text{Says Yes} \mid \text{Comes}) = 1.0$
- Actual attendance rate: $P(\text{Comes}) = 0.3$
- Confirmation rate: $P(\text{Says Yes}) = 0.8$

**Question:** Given confirmation, compute the probability of actual attendance.

**Using Bayes' theorem:**
$$P(\text{Comes} \mid \text{Says Yes}) = \frac{P(\text{Says Yes} \mid \text{Comes}) \cdot P(\text{Comes})}{P(\text{Says Yes})}$$

$$= \frac{(1.0)(0.3)}{0.8} = \frac{0.3}{0.8} = 0.375 = 37.5\%$$

**Result:** Despite confirmation, the probability of attendance is 37.5%.

**Analysis:** The low posterior probability results from the high base rate of confirmations (80%) relative to the actual attendance rate (30%). Most confirmations do not result in attendance.

---

### 4.6 Example: Email Spam Classification

Email characteristics:
- **Prior:** $P(\text{Spam}) = 0.2$
- **Likelihood:** $P(\text{Contains "FREE MONEY"} \mid \text{Spam}) = 0.9$
- **Marginal:** $P(\text{Contains "FREE MONEY"}) = 0.25$

For an email containing "FREE MONEY", compute:

$$P(\text{Spam} \mid \text{"FREE MONEY"}) = \frac{(0.9)(0.2)}{0.25} = \frac{0.18}{0.25} = 0.72 = 72\%$$

**Result:** The email has a 72% probability of being spam.

**Process:**
1. Prior belief: 20% of emails are spam
2. Evidence observed: "FREE MONEY" present
3. Updated belief (posterior): 72% probability of spam

This exemplifies Bayesian updating, the foundation of spam filters and probabilistic classifiers.

---

## Section 5: Independence

### 5.1 Independence of Events

**Definition 5.1 (Independence):**
Two random variables $A$ and $B$ are **independent** (denoted $A \perp B$) if and only if:
$$P(A, B) = P(A) P(B)$$
for all values of $A$ and $B$.

**Theorem 5.1 (Equivalent Characterization):**
If $P(B) > 0$, then $A \perp B$ if and only if:
$$P(A \mid B) = P(A)$$

**Proof:**

**Direction 1: Independence implies conditional equals marginal**

**Step 1:** Assume $A \perp B$, so $P(A, B) = P(A) P(B)$.

**Step 2:** By definition of conditional probability:
$$P(A \mid B) = \frac{P(A, B)}{P(B)}$$

**Step 3:** Substitute independence:
$$P(A \mid B) = \frac{P(A) P(B)}{P(B)}$$

**Step 4:** Cancel $P(B)$ (valid since $P(B) > 0$):
$$P(A \mid B) = P(A)$$

**Direction 2: Conditional equals marginal implies independence**

**Step 1:** Assume $P(A \mid B) = P(A)$.

**Step 2:** By definition of conditional probability:
$$P(A \mid B) = \frac{P(A, B)}{P(B)}$$

**Step 3:** Substitute assumption:
$$P(A) = \frac{P(A, B)}{P(B)}$$

**Step 4:** Multiply both sides by $P(B)$:
$$P(A) P(B) = P(A, B)$$

Therefore $A \perp B$. 

**Interpretation:** Independence means that knowing $B$ provides no information about $A$ - the conditional probability equals the unconditional probability.

### 5.2 Conditional Independence

**Definition 5.2 (Conditional Independence):**
Random variables $A$ and $B$ are **conditionally independent given $C$** (denoted $A \perp B \mid C$) if and only if:
$$P(A, B \mid C) = P(A \mid C) P(B \mid C)$$
for all values of $A$, $B$, and $C$ (where $P(C) > 0$).

**Theorem 5.2 (Equivalent Characterization):**
If $P(B, C) > 0$, then $A \perp B \mid C$ if and only if:
$$P(A \mid B, C) = P(A \mid C)$$

**Proof:**

**Direction 1:**

**Step 1:** Assume $A \perp B \mid C$, so $P(A, B \mid C) = P(A \mid C) P(B \mid C)$.

**Step 2:** By definition of conditional probability:
$$P(A \mid B, C) = \frac{P(A, B \mid C)}{P(B \mid C)}$$

**Step 3:** Substitute conditional independence:
$$P(A \mid B, C) = \frac{P(A \mid C) P(B \mid C)}{P(B \mid C)} = P(A \mid C)$$

**Direction 2:** Similar (reverse the steps). 

**Important Notes:**
- Variables can be **marginally independent** ($A \perp B$) but **conditionally dependent** ($A \not\perp B \mid C$)
- Variables can be **marginally dependent** ($A \not\perp B$) but **conditionally independent** ($A \perp B \mid C$)
- Example of first case: Common effect (explaining away)
- Example of second case: Common cause (confounding)

---

## Section 6: Expectation and Variance

### 6.1 Expectation (Expected Value)

**Definition 6.1 (Expectation - Discrete Case):**
For a discrete random variable $X$ with probability mass function $P(X = x)$, the **expectation** (or expected value, or mean) is:
$$E[X] = \sum_{x} x \cdot P(X = x)$$
where the sum is over all possible values of $X$.

**Conditions for existence:** The expectation exists if and only if $\sum_{x} |x| \cdot P(X = x) < \infty$.

**Alternative Notation:** $E[X] = E_{X \sim P}[X] = \mu_X = \mu$

**Definition 6.2 (Expectation of Function):**
For a function $f: \mathbb{R} \to \mathbb{R}$ and discrete random variable $X$:
$$E_{X \sim P}[f(X)] = \sum_{x} f(x) \cdot P(X = x)$$

**Definition 6.3 (Expectation - Continuous Case):**
For a continuous random variable $X$ with probability density function $f(x)$:
$$E[X] = \int_{-\infty}^{\infty} x \cdot f(x) \, dx$$

For a function $g$:
$$E[g(X)] = \int_{-\infty}^{\infty} g(x) \cdot f(x) \, dx$$

### 6.2 Properties of Expectation

**Theorem 6.1 (Linearity of Expectation):**
For random variables $X$ and $Y$ and constants $a, b \in \mathbb{R}$:
$$E[aX + bY] = aE[X] + bE[Y]$$

**Proof (Discrete Case):**

**Step 1:** Write the expectation:
$$E[aX + bY] = \sum_{x, y} (ax + by) \cdot P(X = x, Y = y)$$

**Step 2:** Distribute:
$$= \sum_{x, y} ax \cdot P(X = x, Y = y) + \sum_{x, y} by \cdot P(X = x, Y = y)$$

**Step 3:** Factor out constants:
$$= a \sum_{x, y} x \cdot P(X = x, Y = y) + b \sum_{x, y} y \cdot P(X = x, Y = y)$$

**Step 4:** For the first term, marginalize over $y$:
$$\sum_{x, y} x \cdot P(X = x, Y = y) = \sum_{x} x \sum_{y} P(X = x, Y = y) = \sum_{x} x \cdot P(X = x) = E[X]$$

**Step 5:** Similarly for the second term:
$$\sum_{x, y} y \cdot P(X = x, Y = y) = E[Y]$$

**Step 6:** Combine:
$$E[aX + bY] = aE[X] + bE[Y]$$

**Important Note:** Linearity holds **regardless of whether $X$ and $Y$ are independent**. This is a powerful property.

**Corollary 6.1.1:** For constants $a$ and $b$:
- $E[aX] = aE[X]$
- $E[X + b] = E[X] + b$
- $E[a] = a$

### 6.3 Variance

**Definition 6.4 (Variance):**
The **variance** of a random variable $X$ measures the spread of its distribution around the mean:
$$\text{Var}[X] = E\left[(X - E[X])^2\right]$$

**Alternative Notation:** $\text{Var}[X] = \sigma_X^2 = \sigma^2$

**Theorem 6.2 (Computational Formula for Variance):**
$$\text{Var}[X] = E[X^2] - (E[X])^2$$

**Proof:**

**Step 1:** Start with definition:
$$\text{Var}[X] = E\left[(X - E[X])^2\right]$$

**Step 2:** Let $\mu = E[X]$ for notational simplicity:
$$\text{Var}[X] = E\left[(X - \mu)^2\right]$$

**Step 3:** Expand the square:
$$= E\left[X^2 - 2\mu X + \mu^2\right]$$

**Step 4:** Apply linearity of expectation:
$$= E[X^2] - E[2\mu X] + E[\mu^2]$$

**Step 5:** Factor out constants:
$$= E[X^2] - 2\mu E[X] + \mu^2$$

**Step 6:** Substitute $\mu = E[X]$:
$$= E[X^2] - 2E[X] \cdot E[X] + (E[X])^2$$

**Step 7:** Simplify:
$$= E[X^2] - 2(E[X])^2 + (E[X])^2$$

**Step 8:** Combine like terms:
$$= E[X^2] - (E[X])^2$$

Therefore $\text{Var}[X] = E[X^2] - (E[X])^2$. 

**Interpretation:** Variance is the difference between the "mean of squares" and "square of mean."

### 6.4 Properties of Variance

**Theorem 6.3 (Variance of Scaled Random Variable):**
For any constant $c$:
$$\text{Var}[cX] = c^2 \text{Var}[X]$$

**Proof:**

**Step 1:** Use the computational formula:
$$\text{Var}[cX] = E[(cX)^2] - (E[cX])^2$$

**Step 2:** Simplify:
$$= E[c^2 X^2] - (cE[X])^2$$
$$= c^2 E[X^2] - c^2 (E[X])^2$$
$$= c^2 (E[X^2] - (E[X])^2)$$
$$= c^2 \text{Var}[X]$$

**Theorem 6.4 (Variance is Shift-Invariant):**
For any constant $c$:
$$\text{Var}[X + c] = \text{Var}[X]$$

**Proof:**

**Step 1:** Note that $E[X + c] = E[X] + c$.

**Step 2:** Apply definition:
$$\text{Var}[X + c] = E[(X + c - E[X + c])^2] = E[(X + c - E[X] - c)^2] = E[(X - E[X])^2] = \text{Var}[X]$$

**Theorem 6.5 (Variance of Sum - Independent Case):**
If $X$ and $Y$ are **independent**, then:
$$\text{Var}[X + Y] = \text{Var}[X] + \text{Var}[Y]$$

**Note:** This does NOT hold in general for dependent random variables.

### 6.5 Standard Deviation

**Definition 6.5 (Standard Deviation):**
The **standard deviation** is the square root of variance:
$$\sigma_X = \sqrt{\text{Var}[X]}$$

**Purpose:** Standard deviation is expressed in the same units as the original random variable, making it more interpretable than variance.

**Example:** If $X$ represents height in centimeters:
- $\text{Var}[X]$ has units of cm² (squared centimeters)
- $\sigma_X$ has units of cm (centimeters)

**Properties:**
- $\sigma_X \geq 0$
- $\sigma_{cX} = |c| \sigma_X$
- $\sigma_{X+c} = \sigma_X$

---

## Section 7: Vector-Valued Random Variables

### 7.1 Expectation of Random Vectors

**Definition 7.1 (Random Vector):**
A **random vector** $\mathbf{x} \in \mathbb{R}^n$ is a vector whose components are random variables:
$$\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{bmatrix}$$

**Definition 7.2 (Expectation of Random Vector):**
The expectation of a random vector is computed component-wise:
$$\boldsymbol{\mu} = E[\mathbf{x}] = \begin{bmatrix} E[x_1] \\ E[x_2] \\ \vdots \\ E[x_n] \end{bmatrix}$$

More explicitly:
$$\mu_i = E[x_i]$$
for $i = 1, 2, \ldots, n$.

### 7.2 Covariance and Covariance Matrix

**Definition 7.3 (Covariance):**
The **covariance** between two random variables $X$ and $Y$ is:
$$\text{Cov}(X, Y) = E[(X - E[X])(Y - E[Y])] = E[XY] - E[X]E[Y]$$

**Properties:**
- $\text{Cov}(X, X) = \text{Var}(X)$
- $\text{Cov}(X, Y) = \text{Cov}(Y, X)$ (symmetric)
- If $X \perp Y$, then $\text{Cov}(X, Y) = 0$

**Definition 7.4 (Covariance Matrix):**
For a random vector $\mathbf{x} \in \mathbb{R}^n$ with mean $\boldsymbol{\mu} = E[\mathbf{x}]$, the **covariance matrix** is:
$$\boldsymbol{\Sigma} = \text{Cov}[\mathbf{x}] = E\left[(\mathbf{x} - \boldsymbol{\mu})(\mathbf{x} - \boldsymbol{\mu})^T\right]$$

**Expanded Form:**
$$\boldsymbol{\Sigma} = \begin{bmatrix}
\text{Var}[x_1] & \text{Cov}[x_1, x_2] & \cdots & \text{Cov}[x_1, x_n] \\
\text{Cov}[x_2, x_1] & \text{Var}[x_2] & \cdots & \text{Cov}[x_2, x_n] \\
\vdots & \vdots & \ddots & \vdots \\
\text{Cov}[x_n, x_1] & \text{Cov}[x_n, x_2] & \cdots & \text{Var}[x_n]
\end{bmatrix}$$

**Properties:**
- Diagonal elements are variances: $\Sigma_{ii} = \text{Var}[x_i]$
- Off-diagonal elements are covariances: $\Sigma_{ij} = \text{Cov}[x_i, x_j]$
- The matrix is symmetric: $\Sigma_{ij} = \Sigma_{ji}$
- The matrix is positive semi-definite

### 7.3 Variance of Linear Combinations

**Theorem 7.1 (Variance of Linear Combination):**
For any constant vector $\mathbf{v} \in \mathbb{R}^n$ and random vector $\mathbf{x}$:
$$\text{Var}[\mathbf{v}^T \mathbf{x}] = \mathbf{v}^T \boldsymbol{\Sigma} \mathbf{v}$$

**Proof:**

**Step 1:** Let $Y = \mathbf{v}^T \mathbf{x}$ be a scalar random variable.

**Step 2:** Compute the mean of $Y$:
$$E[Y] = E[\mathbf{v}^T \mathbf{x}] = \mathbf{v}^T E[\mathbf{x}] = \mathbf{v}^T \boldsymbol{\mu}$$

**Step 3:** Therefore:
$$Y - E[Y] = \mathbf{v}^T \mathbf{x} - \mathbf{v}^T \boldsymbol{\mu} = \mathbf{v}^T (\mathbf{x} - \boldsymbol{\mu})$$

**Step 4:** Compute variance:
$$\text{Var}[Y] = E\left[(Y - E[Y])^2\right] = E\left[(\mathbf{v}^T (\mathbf{x} - \boldsymbol{\mu}))^2\right]$$

**Step 5:** Note that $(\mathbf{v}^T (\mathbf{x} - \boldsymbol{\mu}))^2 = \mathbf{v}^T (\mathbf{x} - \boldsymbol{\mu})(\mathbf{x} - \boldsymbol{\mu})^T \mathbf{v}$:
$$= E\left[\mathbf{v}^T (\mathbf{x} - \boldsymbol{\mu})(\mathbf{x} - \boldsymbol{\mu})^T \mathbf{v}\right]$$

**Step 6:** Factor out constants:
$$= \mathbf{v}^T E\left[(\mathbf{x} - \boldsymbol{\mu})(\mathbf{x} - \boldsymbol{\mu})^T\right] \mathbf{v}$$

**Step 7:** Recognize the covariance matrix:
$$= \mathbf{v}^T \boldsymbol{\Sigma} \mathbf{v}$$

**Important Consequence:** Since variance is always non-negative, we have $\mathbf{v}^T \boldsymbol{\Sigma} \mathbf{v} \geq 0$ for all $\mathbf{v}$, which proves that $\boldsymbol{\Sigma}$ is positive semi-definite.

### 7.4 Covariance and Independence

**Theorem 7.2 (Independence Implies Zero Covariance):**
If $X \perp Y$ (i.e., $X$ and $Y$ are independent), then $\text{Cov}(X, Y) = 0$.

**Proof:**

**Step 1:** By definition of covariance:
$$\text{Cov}(X, Y) = E[XY] - E[X]E[Y]$$

**Step 2:** If $X \perp Y$, then:
$$E[XY] = E[X]E[Y]$$

**Step 3:** Substitute into Step 1:
$$\text{Cov}(X, Y) = E[X]E[Y] - E[X]E[Y] = 0$$

**Important Note:** The converse is NOT true in general. Zero covariance does not imply independence.

**Counterexample:** Let $X \sim \text{Uniform}(-1, 1)$ and $Y = X^2$. Then:
- $E[X] = 0$
- $E[XY] = E[X \cdot X^2] = E[X^3] = 0$ (odd function)
- $\text{Cov}(X, Y) = E[XY] - E[X]E[Y] = 0 - 0 = 0$

But $X$ and $Y$ are clearly dependent (knowing $X$ determines $Y$ exactly)!

**Exception:** For jointly Gaussian random variables, zero covariance DOES imply independence.

---

## Section 8: Practical Examples

### 8.1 HIV Test Example - Complete Calculation

**Problem Setup:**

**Given Information:**
- Let $H = 1$ denote "has HIV", $H = 0$ denote "does not have HIV"
- Let $D_1 = 1$ denote "first test is positive", $D_1 = 0$ denote "first test is negative"
- Prior probability of having HIV: $P(H = 1) = 0.0015$
- Consequently: $P(H = 0) = 1 - 0.0015 = 0.9985$
- Test sensitivity (true positive rate): $P(D_1 = 1 \mid H = 1) = 1.0$
- False positive rate: $P(D_1 = 1 \mid H = 0) = 0.01$

**Question:** What is the probability of having HIV given a positive test result, i.e., $P(H = 1 \mid D_1 = 1)$?

#### 8.1.1 Solution - Single Test

**Step 1:** Apply Bayes' theorem:
$$P(H = 1 \mid D_1 = 1) = \frac{P(D_1 = 1 \mid H = 1) P(H = 1)}{P(D_1 = 1)}$$

**Step 2:** Compute $P(D_1 = 1)$ using marginalization:
$$P(D_1 = 1) = P(D_1 = 1 \mid H = 0) P(H = 0) + P(D_1 = 1 \mid H = 1) P(H = 1)$$

**Step 3:** Substitute values:
$$P(D_1 = 1) = (0.01)(0.9985) + (1.0)(0.0015)$$

**Step 4:** Compute:
$$P(D_1 = 1) = 0.009985 + 0.0015 = 0.011485$$

**Step 5:** Compute the numerator:
$$P(D_1 = 1 \mid H = 1) P(H = 1) = (1.0)(0.0015) = 0.0015$$

**Step 6:** Apply Bayes' theorem:
$$P(H = 1 \mid D_1 = 1) = \frac{0.0015}{0.011485} = 0.1306$$

**Conclusion:** The probability of actually having HIV given a positive test is approximately **13.06%** or about **1 in 8**.

**Interpretation:** Despite a positive test, the probability of actually having the disease is relatively low because:
1. The disease is rare (low prior: 0.15%)
2. The false positive rate (1%) is much higher than the disease prevalence
3. Most positive tests are false positives, not true positives

#### 8.1.2 Solution - Two Tests

**Extended Problem:**
A second test is administered with properties:
- Sensitivity: $P(D_2 = 1 \mid H = 1) = 0.98$
- False positive rate: $P(D_2 = 1 \mid H = 0) = 0.03$

**Assumption:** The tests are **conditionally independent given $H$**, meaning:
$$P(D_1 = 1, D_2 = 1 \mid H) = P(D_1 = 1 \mid H) P(D_2 = 1 \mid H)$$

**Question:** What is $P(H = 1 \mid D_1 = 1, D_2 = 1)$?

**Solution:**

**Step 1:** Apply Bayes' theorem:
$$P(H = 1 \mid D_1 = 1, D_2 = 1) = \frac{P(D_1 = 1, D_2 = 1 \mid H = 1) P(H = 1)}{P(D_1 = 1, D_2 = 1)}$$

**Step 2:** Use conditional independence for $H = 1$:
$$P(D_1 = 1, D_2 = 1 \mid H = 1) = (1.0)(0.98) = 0.98$$

**Step 3:** Use conditional independence for $H = 0$:
$$P(D_1 = 1, D_2 = 1 \mid H = 0) = (0.01)(0.03) = 0.0003$$

**Step 4:** Compute marginal probability:
$$P(D_1 = 1, D_2 = 1) = (0.0003)(0.9985) + (0.98)(0.0015)$$
$$= 0.00029955 + 0.00147 = 0.00176955$$

**Step 5:** Compute numerator:
$$P(D_1 = 1, D_2 = 1 \mid H = 1) P(H = 1) = (0.98)(0.0015) = 0.00147$$

**Step 6:** Apply Bayes' theorem:
$$P(H = 1 \mid D_1 = 1, D_2 = 1) = \frac{0.00147}{0.00176955} = 0.8307$$

**Conclusion:** With two positive tests, the probability of having HIV increases to approximately **83.07%**.

**Interpretation:** The second positive test provides significant additional evidence, increasing confidence from 13.06% to 83.07%.

### 8.2 Investment Variance Example

**Problem Setup:**

An investment has three possible outcomes:
- Return nothing (0×): probability 0.5
- Double investment (2×): probability 0.4
- Tenfold return (10×): probability 0.1

Let $X$ be the return multiplier.

**Question:** Compute $E[X]$ and $\text{Var}[X]$.

#### 8.2.1 Expected Return

$$E[X] = \sum_{x} x \cdot P(X = x)$$
$$= (0)(0.5) + (2)(0.4) + (10)(0.1)$$
$$= 0 + 0.8 + 1.0 = 1.8$$

**Conclusion:** The expected return is **1.8×** the investment (80% gain on average).

#### 8.2.2 Variance of Return

**Step 1:** Compute $E[X^2]$:
$$E[X^2] = (0^2)(0.5) + (2^2)(0.4) + (10^2)(0.1)$$
$$= 0 + 1.6 + 10 = 11.6$$

**Step 2:** Compute variance:
$$\text{Var}[X] = E[X^2] - (E[X])^2 = 11.6 - (1.8)^2 = 11.6 - 3.24 = 8.36$$

**Step 3:** Standard deviation:
$$\sigma = \sqrt{8.36} \approx 2.89$$

**Conclusion:** The variance is **8.36**, indicating high risk.

**Interpretation:**
- The large variance relative to the mean indicates significant uncertainty
- Most likely outcome (50% chance) is losing everything
- Occasional large gains (10× return with 10% probability) pull the expected value up

---

## Section 9: Important Inequalities and Limit Theorems

### 9.1 Chebyshev's Inequality

**Theorem 9.1 (Chebyshev's Inequality):**
For any random variable $X$ with mean $\mu$ and variance $\sigma^2$, and for any $k > 0$:
$$P(|X - \mu| \geq k\sigma) \leq \frac{1}{k^2}$$

**Equivalently:** For any $\epsilon > 0$:
$$P(|X - \mu| \geq \epsilon) \leq \frac{\sigma^2}{\epsilon^2}$$

**Proof:**

**Step 1:** Define an indicator random variable:
$$I = \begin{cases} 1 & \text{if } |X - \mu| \geq \epsilon \\ 0 & \text{otherwise} \end{cases}$$

**Step 2:** Note that $E[I] = P(|X - \mu| \geq \epsilon)$.

**Step 3:** Observe that for all outcomes:
$$I \leq \frac{(X - \mu)^2}{\epsilon^2}$$

*Justification:*
- If $|X - \mu| \geq \epsilon$: Then $(X - \mu)^2 \geq \epsilon^2$, so $\frac{(X - \mu)^2}{\epsilon^2} \geq 1 = I$
- If $|X - \mu| < \epsilon$: Then $I = 0$ and $\frac{(X - \mu)^2}{\epsilon^2} \geq 0 = I$

**Step 4:** Take expectations:
$$E[I] \leq E\left[\frac{(X - \mu)^2}{\epsilon^2}\right] = \frac{\sigma^2}{\epsilon^2}$$

**Step 5:** Therefore:
$$P(|X - \mu| \geq \epsilon) \leq \frac{\sigma^2}{\epsilon^2}$$

**Interpretation:**
- For $k = 2$: At least 75% of values lie within 2 standard deviations of the mean
- For $k = 3$: At least 88.9% of values lie within 3 standard deviations of the mean

**Note:** This bound applies to ANY distribution with finite variance, but is often loose for specific distributions.

### 9.2 Law of Large Numbers

**Theorem 9.2 (Weak Law of Large Numbers):**
Let $X_1, X_2, \ldots, X_n$ be independent and identically distributed (i.i.d.) random variables with mean $\mu$ and finite variance $\sigma^2$. Then the sample average
$$\bar{X}_n = \frac{1}{n} \sum_{i=1}^n X_i$$
converges in probability to $\mu$ as $n \to \infty$:
$$\lim_{n \to \infty} P(|\bar{X}_n - \mu| > \epsilon) = 0 \quad \text{for all } \epsilon > 0$$

**Proof Sketch:**

**Step 1:** Compute the mean of $\bar{X}_n$:
$$E[\bar{X}_n] = \frac{1}{n} \sum_{i=1}^n E[X_i] = \frac{1}{n} \cdot n\mu = \mu$$

**Step 2:** Compute the variance of $\bar{X}_n$:
$$\text{Var}[\bar{X}_n] = \frac{1}{n^2} \sum_{i=1}^n \text{Var}[X_i] = \frac{n\sigma^2}{n^2} = \frac{\sigma^2}{n}$$

**Step 3:** Apply Chebyshev's inequality:
$$P(|\bar{X}_n - \mu| \geq \epsilon) \leq \frac{\text{Var}[\bar{X}_n]}{\epsilon^2} = \frac{\sigma^2}{n\epsilon^2}$$

**Step 4:** Take the limit:
$$\lim_{n \to \infty} P(|\bar{X}_n - \mu| \geq \epsilon) \leq \lim_{n \to \infty} \frac{\sigma^2}{n\epsilon^2} = 0$$

**Interpretation:**
- Empirical averages converge to theoretical means
- Foundation for statistical inference
- Justifies using sample means to estimate population means

### 9.3 Central Limit Theorem

**Theorem 9.3 (Central Limit Theorem):**
Let $X_1, X_2, \ldots, X_n$ be i.i.d. random variables with mean $\mu$ and variance $\sigma^2 < \infty$. Then the standardized sample average
$$Z_n = \frac{\bar{X}_n - \mu}{\sigma/\sqrt{n}} = \frac{\sqrt{n}(\bar{X}_n - \mu)}{\sigma}$$
converges in distribution to a standard normal distribution $N(0,1)$ as $n \to \infty$.

**Equivalently:**
$$\bar{X}_n \approx N\left(\mu, \frac{\sigma^2}{n}\right) \text{ for large } n$$

**Key Insights:**

1. **Convergence Rate:** The standard error is $\sigma/\sqrt{n}$, meaning:
   - To halve the error, need 4× more samples
   - To reduce error by factor of 10, need 100× more samples

2. **Universality:** The CLT applies regardless of the original distribution (as long as variance exists)

3. **Rule of Thumb:** CLT approximation is usually good for $n \geq 30$

**Applications:**
- Confidence intervals
- Hypothesis testing
- Monte Carlo estimation

---

## Section 10: Types of Uncertainty

### 10.1 Aleatoric Uncertainty

**Definition 10.1 (Aleatoric Uncertainty):**
**Aleatoric uncertainty** (also called **irreducible uncertainty** or **stochastic uncertainty**) is the inherent randomness in a system that cannot be reduced by collecting more data.

**Etymology:** From Latin "aleator" (dice player)

**Examples:**
- Quantum randomness
- Thermal noise in electronic circuits
- Outcome of a fair coin flip

**Characteristics:**
- Intrinsic to the system
- Cannot be eliminated
- Best we can do is characterize the distribution

### 10.2 Epistemic Uncertainty

**Definition 10.2 (Epistemic Uncertainty):**
**Epistemic uncertainty** (also called **model uncertainty** or **reducible uncertainty**) is uncertainty about model parameters or structure that can be reduced by collecting more data.

**Etymology:** From Greek "episteme" (knowledge)

**Examples:**
- Uncertainty in parameter estimates due to limited data
- Model selection uncertainty
- Uncertainty about the true underlying distribution

**Characteristics:**
- Due to lack of knowledge
- Can be reduced with more data
- Captured by distributions over parameters (Bayesian approach)

**Important Distinction:**
- **Aleatoric:** "We'll never know which outcome will occur, even with infinite data"
- **Epistemic:** "We don't know yet, but more data will help"

---

## Section 11: Sampling

### 11.1 Sampling from Distributions

**Motivation:** In practice, we often need to generate random samples from a probability distribution for:
- Monte Carlo estimation
- Simulation studies
- Bayesian inference
- Machine learning (dropout, data augmentation)

**Definition 11.1 (Random Sample):**
A **random sample** of size $n$ from a distribution $P$ is a collection of $n$ i.i.d. random variables $X_1, X_2, \ldots, X_n$ where each $X_i \sim P$.

### 11.2 Inverse Transform Sampling

**Theorem 11.1 (Inverse Transform):**
If $U \sim \text{Uniform}(0,1)$ and $F$ is a CDF with inverse $F^{-1}$, then:
$$X = F^{-1}(U) \sim F$$

**Proof:**

**Step 1:** We need to show $P(X \leq x) = F(x)$.

**Step 2:** Compute:
$$P(X \leq x) = P(F^{-1}(U) \leq x) = P(U \leq F(x))$$

**Step 3:** Since $U \sim \text{Uniform}(0,1)$:
$$P(U \leq F(x)) = F(x)$$

**Algorithm:**
1. Generate $U \sim \text{Uniform}(0,1)$
2. Return $X = F^{-1}(U)$

**Example:** Exponential distribution with rate $\lambda$
- CDF: $F(x) = 1 - e^{-\lambda x}$
- Inverse: $F^{-1}(u) = -\frac{\ln(1-u)}{\lambda}$
- Sample: $X = -\frac{\ln(1-U)}{\lambda}$ where $U \sim \text{Uniform}(0,1)$

---

## Section 12: Sigma-Algebra (Advanced Topic)

### 12.1 Why Do We Need Sigma-Algebras?

**Motivation:** For continuous sample spaces like $\mathbb{R}$, not all subsets can be assigned consistent probabilities. The $\sigma$-algebra defines which events are "measurable."

**Definition 12.1 ($\sigma$-Algebra):**
A **$\sigma$-algebra** $\mathcal{F}$ on a set $\mathcal{S}$ is a collection of subsets satisfying:

1. **Contains sample space:** $\mathcal{S} \in \mathcal{F}$

2. **Closed under complementation:** If $E \in \mathcal{F}$, then $E^c \in \mathcal{F}$

3. **Closed under countable unions:** If $E_1, E_2, E_3, \ldots \in \mathcal{F}$, then $\bigcup_{i=1}^{\infty} E_i \in \mathcal{F}$

**Consequences:**
- $\emptyset \in \mathcal{F}$ (since $\emptyset = \mathcal{S}^c$)
- Closed under countable intersections (by De Morgan's laws)
- Closed under set differences

**Common Examples:**
1. **Trivial $\sigma$-algebra:** $\mathcal{F} = \{\emptyset, \mathcal{S}\}$
2. **Power set:** $\mathcal{F} = 2^{\mathcal{S}}$ (all subsets) - works for countable $\mathcal{S}$
3. **Borel $\sigma$-algebra on $\mathbb{R}$:** Generated by all open intervals

**For practical purposes:** In discrete/finite settings, we can use the power set and ignore these technicalities.

---

## Section 13: Gaussian (Normal) Distribution

### 13.1 Definition

**Definition 13.1 (Gaussian Distribution):**
A random variable $X$ follows a **Gaussian (normal) distribution** with parameters $\mu$ (mean) and $\sigma^2$ (variance), denoted $X \sim \mathcal{N}(\mu, \sigma^2)$, if it has PDF:
$$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$$

**Standard Normal:** When $\mu = 0$ and $\sigma = 1$:
$$\phi(z) = \frac{1}{\sqrt{2\pi}} \exp\left(-\frac{z^2}{2}\right)$$

### 13.2 Key Properties

1. **Symmetry:** $f(\mu + x) = f(\mu - x)$
2. **Mean:** $E[X] = \mu$
3. **Variance:** $\text{Var}(X) = \sigma^2$
4. **Standardization:** If $X \sim \mathcal{N}(\mu, \sigma^2)$, then $Z = \frac{X - \mu}{\sigma} \sim \mathcal{N}(0, 1)$

### 13.3 The 68-95-99.7 Rule

For a normal distribution:
- Approximately 68% of values lie within $\mu \pm \sigma$
- Approximately 95% of values lie within $\mu \pm 2\sigma$
- Approximately 99.7% of values lie within $\mu \pm 3\sigma$

### 13.4 Proof that the PDF Integrates to 1

**Claim:** $\int_{-\infty}^{\infty} \frac{1}{\sqrt{2\pi}} e^{-z^2/2} dz = 1$

**Proof:**

**Step 1:** Let $I = \int_{-\infty}^{\infty} e^{-z^2/2} dz$. We need to show $I = \sqrt{2\pi}$.

**Step 2:** Compute $I^2$:
$$I^2 = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} e^{-(z^2 + w^2)/2} dz \, dw$$

**Step 3:** Convert to polar coordinates $(r, \theta)$:
$$I^2 = \int_0^{2\pi} \int_0^{\infty} e^{-r^2/2} r \, dr \, d\theta$$

**Step 4:** Evaluate the radial integral with substitution $u = r^2/2$:
$$\int_0^{\infty} e^{-r^2/2} r \, dr = \int_0^{\infty} e^{-u} du = 1$$

**Step 5:** Complete the calculation:
$$I^2 = \int_0^{2\pi} 1 \, d\theta = 2\pi$$

**Step 6:** Therefore $I = \sqrt{2\pi}$.



---

## Section 14: Bernoulli Distribution

### 14.1 Definition

**Definition 14.1 (Bernoulli Distribution):**
A random variable $X$ follows a **Bernoulli distribution** with parameter $p$, denoted $X \sim \text{Bernoulli}(p)$, if:
$$P(X = 1) = p, \quad P(X = 0) = 1 - p$$

where $0 \leq p \leq 1$.

### 14.2 Mean and Variance

**Theorem 14.1:**
For $X \sim \text{Bernoulli}(p)$:
- $E[X] = p$
- $\text{Var}(X) = p(1-p)$

**Proof of Mean:**
$$E[X] = 0 \cdot (1-p) + 1 \cdot p = p$$

**Proof of Variance:**

**Step 1:** Note that $X^2 = X$ since $X \in \{0, 1\}$.

**Step 2:** Therefore $E[X^2] = E[X] = p$.

**Step 3:** Apply variance formula:
$$\text{Var}(X) = E[X^2] - (E[X])^2 = p - p^2 = p(1-p)$$

**Properties:**
- Maximum variance at $p = 0.5$: $\text{Var}(X) = 0.25$
- Variance is symmetric: same for $p$ and $1-p$
- Variance is 0 when $p = 0$ or $p = 1$ (deterministic outcomes)

---

## Section 15: Verification and Summary

### 15.1 Cross-Verification Against Authoritative Sources

All formulas and proofs in this notebook have been verified against:

1. **D2L Textbook** (https://d2l.ai/chapter_preliminaries/probability.html)
2. **Wikipedia** articles on probability theory
3. **Standard textbooks:**
   - Sheldon Ross, "A First Course in Probability"
   - Dimitri Bertsekas and John Tsitsiklis, "Introduction to Probability"

### 15.2 Key Formulas Summary

| Concept | Formula | Conditions |
|---------|---------|------------|
| Conditional Probability | $P(A \mid B) = \frac{P(A,B)}{P(B)}$ | $P(B) > 0$ |
| Bayes' Theorem | $P(A \mid B) = \frac{P(B \mid A)P(A)}{P(B)}$ | $P(A), P(B) > 0$ |
| Marginalization | $P(A) = \sum_b P(A,B=b)$ | - |
| Independence | $P(A,B) = P(A)P(B)$ | Definition |
| Expectation (discrete) | $E[X] = \sum_x x \cdot P(X=x)$ | Sum converges |
| Variance | $\text{Var}(X) = E[X^2] - (E[X])^2$ | Variance exists |
| Linearity of Expectation | $E[aX+bY] = aE[X] + bE[Y]$ | Always holds |
| Chebyshev's Inequality | $P(|X-\mu| \geq k\sigma) \leq 1/k^2$ | $\sigma^2 < \infty$ |
| Bernoulli Mean | $E[X] = p$ | $X \sim \text{Bernoulli}(p)$ |
| Bernoulli Variance | $\text{Var}(X) = p(1-p)$ | $X \sim \text{Bernoulli}(p)$ |

### 15.3 Notation Summary

| Symbol | Meaning |
|--------|--------|
| $\mathcal{S}$ or $\Omega$ | Sample space |
| $P(A)$ | Probability of event $A$ |
| $P(A,B)$ | Joint probability |
| $P(A \mid B)$ | Conditional probability |
| $E[X]$ or $\mu$ | Expected value |
| $\text{Var}(X)$ or $\sigma^2$ | Variance |
| $\sigma$ | Standard deviation |
| $A \perp B$ | $A$ independent of $B$ |
| $A \perp B \mid C$ | $A$ conditionally independent of $B$ given $C$ |
| $\boldsymbol{\Sigma}$ | Covariance matrix |
| $\mathcal{N}(\mu, \sigma^2)$ | Normal distribution |

---

## References

1. Zhang, A., Lipton, Z. C., Li, M., & Smola, A. J. (2021). Dive into Deep Learning. https://d2l.ai/

2. Ross, S. M. (2014). A First Course in Probability (9th ed.). Pearson.

3. Bertsekas, D. P., & Tsitsiklis, J. N. (2008). Introduction to Probability (2nd ed.). Athena Scientific.

4. Wikipedia contributors. (2024). Probability axioms. Wikipedia. https://en.wikipedia.org/wiki/Probability_axioms

5. Wikipedia contributors. (2024). Bayes' theorem. Wikipedia. https://en.wikipedia.org/wiki/Bayes%27_theorem

In [ ]:
# Exercise 8: Portfolio Optimization
from scipy.optimize import minimize

# Given data
mu = np.array([0.10, 0.08, 0.12])  # Expected returns
Sigma = np.array([
    [0.04, 0.01, 0.02],
    [0.01, 0.02, 0.005],
    [0.02, 0.005, 0.06]
])

asset_names = ['Asset A', 'Asset B', 'Asset C']

print("="*70)
print("EXERCISE 8: PORTFOLIO OPTIMIZATION")
print("="*70)

print("\nGiven Data:")
print("-"*70)
print(f"Expected Returns: {mu}")
print(f"\nCovariance Matrix:")
print(Sigma)

# Part (a): Equal-weighted portfolio
print("\n" + "="*70)
print("Part (a): Equal-Weighted Portfolio")
print("-"*70)

w_equal = np.array([1/3, 1/3, 1/3])
return_equal = w_equal @ mu
variance_equal = w_equal @ Sigma @ w_equal
std_equal = np.sqrt(variance_equal)

print(f"Weights: {w_equal}")
print(f"Expected Return: {return_equal:.4f} = {return_equal*100:.2f}%")
print(f"Variance: {variance_equal:.4f}")
print(f"Standard Deviation: {std_equal:.4f} = {std_equal*100:.2f}%")

# Part (b): Maximum return portfolio
print("\n" + "="*70)
print("Part (b): Maximum Return Portfolio (No Risk Constraint)")
print("-"*70)

max_return_idx = np.argmax(mu)
w_max_return = np.zeros(3)
w_max_return[max_return_idx] = 1.0

return_max = w_max_return @ mu
variance_max = w_max_return @ Sigma @ w_max_return
std_max = np.sqrt(variance_max)

print(f"Optimal Allocation: 100% to {asset_names[max_return_idx]}")
print(f"Weights: {w_max_return}")
print(f"Expected Return: {return_max:.4f} = {return_max*100:.2f}%")
print(f"Variance: {variance_max:.4f}")
print(f"Standard Deviation: {std_max:.4f} = {std_max*100:.2f}%")

# Part (c): Variance for different portfolios
print("\n" + "="*70)
print("Part (c): Variance Analysis for Different Portfolios")
print("-"*70)

portfolios = {
    'Equal-weighted': w_equal,
    'Max Return': w_max_return,
    'Conservative (50% B, 50% A)': np.array([0.5, 0.5, 0.0]),
    'Aggressive (50% C, 30% A, 20% B)': np.array([0.3, 0.2, 0.5])
}

print(f"{'Portfolio':<35} {'Return':<10} {'Std Dev':<10}")
print("-"*70)
for name, weights in portfolios.items():
    ret = weights @ mu
    std = np.sqrt(weights @ Sigma @ weights)
    print(f"{name:<35} {ret:8.2%}   {std:8.2%}")

# Part (d): Optimization with risk constraint
print("\n" + "="*70)
print("Part (d): Constrained Optimization - Efficient Frontier")
print("-"*70)

def portfolio_return(w):
    return -(w @ mu)  # Negative because we minimize

def portfolio_variance(w):
    return w @ Sigma @ w

# Constraint: weights sum to 1
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# Bounds: weights between 0 and 1 (no short selling)
bounds = tuple((0, 1) for _ in range(3))

# Compute efficient frontier
target_returns = np.linspace(mu.min(), mu.max(), 50)
efficient_frontier_std = []
efficient_frontier_weights = []

for target_return in target_returns:
    # Minimize variance subject to target return
    constraints_with_return = constraints + [
        {'type': 'eq', 'fun': lambda w, tr=target_return: w @ mu - tr}
    ]
    
    result = minimize(portfolio_variance, x0=w_equal, method='SLSQP',
                     bounds=bounds, constraints=constraints_with_return)
    
    if result.success:
        efficient_frontier_std.append(np.sqrt(result.fun))
        efficient_frontier_weights.append(result.x)
    else:
        efficient_frontier_std.append(np.nan)
        efficient_frontier_weights.append(None)

efficient_frontier_std = np.array(efficient_frontier_std)

# Optimize for different risk aversion parameters
lambdas = [0.5, 1.0, 2.0, 5.0, 10.0]
optimal_portfolios = []

for lam in lambdas:
    # Maximize risk-adjusted return: return - (lambda/2)*variance
    def objective(w):
        return -(w @ mu - (lam/2) * (w @ Sigma @ w))
    
    result = minimize(objective, x0=w_equal, method='SLSQP',
                     bounds=bounds, constraints=constraints)
    
    if result.success:
        w_opt = result.x
        ret_opt = w_opt @ mu
        std_opt = np.sqrt(w_opt @ Sigma @ w_opt)
        optimal_portfolios.append((lam, w_opt, ret_opt, std_opt))

print("\nOptimal Portfolios for Different Risk Aversion Levels:")
print(f"{'Lambda':<8} {'Return':<10} {'Std Dev':<10} {'Weights (A, B, C)'}")
print("-"*70)
for lam, w, ret, std in optimal_portfolios:
    weights_str = f"({w[0]:.3f}, {w[1]:.3f}, {w[2]:.3f})"
    print(f"{lam:<8.1f} {ret:8.2%}   {std:8.2%}   {weights_str}")

# Visualization
fig = plt.figure(figsize=(16, 5))

# Plot 1: Efficient Frontier
ax1 = plt.subplot(1, 3, 1)

# Plot efficient frontier
valid_mask = ~np.isnan(efficient_frontier_std)
ax1.plot(efficient_frontier_std[valid_mask], target_returns[valid_mask], 
         'b-', linewidth=2, label='Efficient Frontier')

# Plot individual assets
for i, name in enumerate(asset_names):
    w_single = np.zeros(3)
    w_single[i] = 1.0
    ret_single = mu[i]
    std_single = np.sqrt(Sigma[i, i])
    ax1.plot(std_single, ret_single, 'ro', markersize=10)
    ax1.annotate(name, (std_single, ret_single), 
                xytext=(5, 5), textcoords='offset points')

# Plot example portfolios
for lam, w, ret, std in optimal_portfolios:
    ax1.plot(std, ret, 'g^', markersize=8)

ax1.plot(std_equal, return_equal, 'ks', markersize=10, label='Equal-weighted')

ax1.set_xlabel('Standard Deviation (Risk)')
ax1.set_ylabel('Expected Return')
ax1.set_title('Efficient Frontier')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Portfolio weights vs lambda
ax2 = plt.subplot(1, 3, 2)

lambdas_plot = [p[0] for p in optimal_portfolios]
weights_A = [p[1][0] for p in optimal_portfolios]
weights_B = [p[1][1] for p in optimal_portfolios]
weights_C = [p[1][2] for p in optimal_portfolios]

ax2.plot(lambdas_plot, weights_A, 'o-', label='Asset A', linewidth=2)
ax2.plot(lambdas_plot, weights_B, 's-', label='Asset B', linewidth=2)
ax2.plot(lambdas_plot, weights_C, '^-', label='Asset C', linewidth=2)

ax2.set_xlabel('Risk Aversion (λ)')
ax2.set_ylabel('Portfolio Weight')
ax2.set_title('Optimal Weights vs Risk Aversion')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Return vs Risk tradeoff
ax3 = plt.subplot(1, 3, 3)

returns_plot = [p[2] for p in optimal_portfolios]
stds_plot = [p[3] for p in optimal_portfolios]

ax3.plot(lambdas_plot, returns_plot, 'b-o', label='Expected Return', linewidth=2)
ax3.set_xlabel('Risk Aversion (λ)')
ax3.set_ylabel('Expected Return', color='b')
ax3.tick_params(axis='y', labelcolor='b')

ax3_twin = ax3.twinx()
ax3_twin.plot(lambdas_plot, stds_plot, 'r-s', label='Standard Deviation', linewidth=2)
ax3_twin.set_ylabel('Standard Deviation', color='r')
ax3_twin.tick_params(axis='y', labelcolor='r')

ax3.set_title('Return-Risk Tradeoff')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("Key Insights:")
print("- Higher risk aversion (λ) → Lower risk, lower return")
print("- Efficient frontier shows optimal return for each risk level")
print("- Diversification reduces risk compared to single-asset portfolios")
print("="*70)

---

## Exercise 8: Portfolio Optimization

> "Portfolio optimization problem with stocks, returns, and covariance."

**Setup:**

Consider a portfolio with three assets:
- Asset A
- Asset B
- Asset C

**Given Data:**

**Expected Returns:**
- $\mu_A = 0.10$ (10% expected return)
- $\mu_B = 0.08$ (8% expected return)
- $\mu_C = 0.12$ (12% expected return)

**Covariance Matrix:**
$$\boldsymbol{\Sigma} = \begin{bmatrix}
0.04 & 0.01 & 0.02 \\
0.01 & 0.02 & 0.005 \\
0.02 & 0.005 & 0.06
\end{bmatrix}$$

**Portfolio Weights:** $\mathbf{w} = (w_A, w_B, w_C)^T$ where $w_A + w_B + w_C = 1$

---

### Part (a): Expected Return for a Given Portfolio

For a portfolio with weights $\mathbf{w}$, the expected return is:
$$E[R_p] = \mathbf{w}^T \boldsymbol{\mu} = w_A \mu_A + w_B \mu_B + w_C \mu_C$$

**Example:** For equal weights $\mathbf{w} = (1/3, 1/3, 1/3)^T$:
$$E[R_p] = \frac{1}{3}(0.10) + \frac{1}{3}(0.08) + \frac{1}{3}(0.12)$$
$$= \frac{1}{3}(0.30) = 0.10 = 10\%$$

**Answer:** Expected return is **10%** for equal-weighted portfolio.

---

### Part (b): Maximum Return Portfolio

To maximize return without constraints on risk:
$$\max_{\mathbf{w}} \quad \mathbf{w}^T \boldsymbol{\mu}$$
$$\text{subject to} \quad \mathbf{w}^T \mathbf{1} = 1$$

**Solution:**

Since Asset C has the highest expected return (12%), allocate all weight to Asset C:
$$\mathbf{w}^* = (0, 0, 1)^T$$

**Maximum Expected Return:**
$$E[R_p^*] = 0 \cdot 0.10 + 0 \cdot 0.08 + 1 \cdot 0.12 = 0.12 = 12\%$$

**Answer:** Invest 100% in Asset C for maximum return of **12%**.

**Caveat:** This ignores risk completely - Asset C may have high variance.

---

### Part (c): Variance of the Portfolio

For portfolio with weights $\mathbf{w}$, variance is:
$$\text{Var}[R_p] = \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$$

Expanding:
$$= \sum_{i,j} w_i w_j \Sigma_{ij}$$

**Example 1: Equal-weighted portfolio** $\mathbf{w} = (1/3, 1/3, 1/3)^T$:

$$\text{Var}[R_p] = \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$$

Computing step-by-step:
$$\boldsymbol{\Sigma} \mathbf{w} = \begin{bmatrix}
0.04 & 0.01 & 0.02 \\
0.01 & 0.02 & 0.005 \\
0.02 & 0.005 & 0.06
\end{bmatrix} \begin{bmatrix} 1/3 \\ 1/3 \\ 1/3 \end{bmatrix} = \begin{bmatrix} 0.0233 \\ 0.0117 \\ 0.0283 \end{bmatrix}$$

$$\mathbf{w}^T (\boldsymbol{\Sigma} \mathbf{w}) = \frac{1}{3}(0.0233 + 0.0117 + 0.0283) = \frac{0.0633}{3} = 0.0211$$

**Answer:** Variance = **0.0211** (standard deviation = **14.5%**)

**Example 2: Maximum return portfolio** $\mathbf{w} = (0, 0, 1)^T$:

$$\text{Var}[R_p] = 1^2 \cdot \Sigma_{CC} = 0.06$$

**Answer:** Variance = **0.06** (standard deviation = **24.5%**)

The maximum return portfolio has much higher risk!

---

### Part (d): Constrained Optimization Problem

**Objective:** Maximize return while limiting risk.

**Formulation:**
$$\max_{\mathbf{w}} \quad \mathbf{w}^T \boldsymbol{\mu}$$
$$\text{subject to} \quad \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w} \leq \sigma_{\max}^2$$
$$\quad \quad \quad \quad \quad \mathbf{w}^T \mathbf{1} = 1$$
$$\quad \quad \quad \quad \quad w_i \geq 0 \quad \forall i$$

where $\sigma_{\max}^2$ is the maximum acceptable variance.

**Alternative Formulation (Risk-Adjusted Return):**
$$\max_{\mathbf{w}} \quad \mathbf{w}^T \boldsymbol{\mu} - \frac{\lambda}{2} \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w}$$
$$\text{subject to} \quad \mathbf{w}^T \mathbf{1} = 1$$

where $\lambda > 0$ is the risk aversion parameter.

**Interpretation:**
- $\lambda = 0$: No risk aversion (maximize return only)
- $\lambda \to \infty$: Extreme risk aversion (minimize variance only)
- Intermediate $\lambda$: Trade-off between return and risk

**Solution Method:**

This is a **quadratic programming** problem. The Lagrangian is:
$$\mathcal{L}(\mathbf{w}, \nu) = \mathbf{w}^T \boldsymbol{\mu} - \frac{\lambda}{2} \mathbf{w}^T \boldsymbol{\Sigma} \mathbf{w} - \nu(\mathbf{w}^T \mathbf{1} - 1)$$

Taking derivative with respect to $\mathbf{w}$ and setting to zero:
$$\boldsymbol{\mu} - \lambda \boldsymbol{\Sigma} \mathbf{w} - \nu \mathbf{1} = 0$$

Solving:
$$\mathbf{w}^* = \frac{1}{\lambda} \boldsymbol{\Sigma}^{-1}(\boldsymbol{\mu} - \nu \mathbf{1})$$

where $\nu$ is chosen to satisfy $\mathbf{w}^T \mathbf{1} = 1$.

**Efficient Frontier:**

The set of portfolios that maximize return for each level of risk forms the **efficient frontier**. Rational investors choose portfolios on this frontier.

In [ ]:
# Exercise 7: Medical Testing with Conditional Dependence
import pandas as pd

# Given probabilities
p_disease = 0.0015
p_healthy = 1 - p_disease

# Test characteristics
p_d1_given_disease = 1.0
p_d1_given_healthy = 0.01
p_d2_given_disease = 0.98

# Conditional dependence for healthy patients
p_d2_given_healthy_and_d1 = 0.30  # If healthy AND test 1 positive
p_d2_given_healthy_and_not_d1 = 0.01  # If healthy AND test 1 negative

# Part (a): Joint probability table for D1, D2 given H=0
print("="*70)
print("EXERCISE 7: MEDICAL TESTING WITH CONDITIONAL DEPENDENCE")
print("="*70)

print("\nPart (a): Joint Probability Table for D1 and D2 given H = 0")
print("-"*70)

# Compute joint probabilities
p_d1_d2_given_h0 = {
    (0, 0): (1 - p_d2_given_healthy_and_not_d1) * (1 - p_d1_given_healthy),
    (0, 1): p_d2_given_healthy_and_not_d1 * (1 - p_d1_given_healthy),
    (1, 0): (1 - p_d2_given_healthy_and_d1) * p_d1_given_healthy,
    (1, 1): p_d2_given_healthy_and_d1 * p_d1_given_healthy
}

# Create DataFrame
df_joint = pd.DataFrame(index=['D1=0', 'D1=1', 'Marginal'], 
                        columns=['D2=0', 'D2=1', 'Marginal'])

df_joint.loc['D1=0', 'D2=0'] = p_d1_d2_given_h0[(0, 0)]
df_joint.loc['D1=0', 'D2=1'] = p_d1_d2_given_h0[(0, 1)]
df_joint.loc['D1=1', 'D2=0'] = p_d1_d2_given_h0[(1, 0)]
df_joint.loc['D1=1', 'D2=1'] = p_d1_d2_given_h0[(1, 1)]

# Compute marginals
df_joint.loc['D1=0', 'Marginal'] = p_d1_d2_given_h0[(0, 0)] + p_d1_d2_given_h0[(0, 1)]
df_joint.loc['D1=1', 'Marginal'] = p_d1_d2_given_h0[(1, 0)] + p_d1_d2_given_h0[(1, 1)]
df_joint.loc['Marginal', 'D2=0'] = p_d1_d2_given_h0[(0, 0)] + p_d1_d2_given_h0[(1, 0)]
df_joint.loc['Marginal', 'D2=1'] = p_d1_d2_given_h0[(0, 1)] + p_d1_d2_given_h0[(1, 1)]
df_joint.loc['Marginal', 'Marginal'] = 1.0

print(df_joint)

# Part (b): One positive test
print("\n" + "="*70)
print("Part (b): Probability of Disease After One Positive Test")
print("-"*70)

p_d1 = p_d1_given_disease * p_disease + p_d1_given_healthy * p_healthy
p_disease_given_d1 = (p_d1_given_disease * p_disease) / p_d1

print(f"P(D1 = 1) = {p_d1:.6f}")
print(f"P(H = 1 | D1 = 1) = {p_disease_given_d1:.4f} = {p_disease_given_d1*100:.2f}%")

# Part (c): Both tests positive
print("\n" + "="*70)
print("Part (c): Probability of Disease After Both Tests Positive")
print("-"*70)

# For diseased patients (assume independence given disease)
p_d1_d2_given_disease = p_d1_given_disease * p_d2_given_disease

# Marginal probability of both tests positive
p_d1_d2 = (p_d1_d2_given_disease * p_disease + 
           p_d1_d2_given_h0[(1, 1)] * p_healthy)

p_disease_given_both = (p_d1_d2_given_disease * p_disease) / p_d1_d2

print(f"P(D1 = 1, D2 = 1 | H = 1) = {p_d1_d2_given_disease:.4f}")
print(f"P(D1 = 1, D2 = 1 | H = 0) = {p_d1_d2_given_h0[(1, 1)]:.4f}")
print(f"P(D1 = 1, D2 = 1) = {p_d1_d2:.6f}")
print(f"\nP(H = 1 | D1 = 1, D2 = 1) = {p_disease_given_both:.4f} = {p_disease_given_both*100:.2f}%")

# Comparison with independent case
p_d2_given_healthy_indep = 0.03
p_d1_d2_given_h0_indep = p_d1_given_healthy * p_d2_given_healthy_indep
p_d1_d2_indep = p_d1_d2_given_disease * p_disease + p_d1_d2_given_h0_indep * p_healthy
p_disease_given_both_indep = (p_d1_d2_given_disease * p_disease) / p_d1_d2_indep

print("\n" + "="*70)
print("Comparison: Dependent vs Independent Tests")
print("-"*70)
print(f"                                    Dependent    Independent")
print(f"P(both positive | healthy):         {p_d1_d2_given_h0[(1, 1)]:.6f}     {p_d1_d2_given_h0_indep:.6f}")
print(f"P(disease | one positive):          {p_disease_given_d1:.4f}       {p_disease_given_d1:.4f}")
print(f"P(disease | both positive):         {p_disease_given_both:.4f}       {p_disease_given_both_indep:.4f}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Progression of posterior probability
ax1 = axes[0]
scenarios = ['Prior\n(no tests)', 'After\n1 positive', 'After both\npositive\n(dependent)', 
             'After both\npositive\n(independent)']
probs = [p_disease, p_disease_given_d1, p_disease_given_both, p_disease_given_both_indep]
colors = ['gray', 'yellow', 'orange', 'red']

bars = ax1.bar(range(len(scenarios)), probs, color=colors, edgecolor='black', linewidth=2)
ax1.set_ylabel('P(Disease | Evidence)')
ax1.set_title('Bayesian Updating with Test Results')
ax1.set_xticks(range(len(scenarios)))
ax1.set_xticklabels(scenarios, fontsize=9)
ax1.set_ylim([0, 1])
ax1.grid(True, alpha=0.3, axis='y')

# Add percentage labels
for i, (bar, prob) in enumerate(zip(bars, probs)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{prob*100:.1f}%', ha='center', va='bottom', fontweight='bold')

# Plot 2: Joint probability heatmap for dependent case
ax2 = axes[1]
joint_matrix = np.array([[p_d1_d2_given_h0[(0, 0)], p_d1_d2_given_h0[(0, 1)]],
                         [p_d1_d2_given_h0[(1, 0)], p_d1_d2_given_h0[(1, 1)]]])

im = ax2.imshow(joint_matrix, cmap='YlOrRd', aspect='auto')
ax2.set_xticks([0, 1])
ax2.set_yticks([0, 1])
ax2.set_xticklabels(['D2=0', 'D2=1'])
ax2.set_yticklabels(['D1=0', 'D1=1'])
ax2.set_title('P(D1, D2 | H=0) - Conditional Dependence')

# Add text annotations
for i in range(2):
    for j in range(2):
        text = ax2.text(j, i, f'{joint_matrix[i, j]:.4f}',
                       ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im, ax=ax2)
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("Key Insight: Conditional dependence reduces evidential value!")
print("When tests are conditionally dependent for healthy patients,")
print("both positive results provide less evidence than if independent.")
print("="*70)

---

## Exercise 7: Medical Testing with Conditional Dependence

> "Tests with modified parameters involving conditional dependence for healthy patients."

**Modification from Section 8:** The two diagnostic tests are now **conditionally dependent** given health status.

**Given:**
- $P(H = 1) = 0.0015$ (prevalence of disease)
- $P(D_1 = 1 \mid H = 1) = 1.0$ (perfect sensitivity for test 1)
- $P(D_1 = 1 \mid H = 0) = 0.01$ (1% false positive for test 1)
- $P(D_2 = 1 \mid H = 1) = 0.98$ (98% sensitivity for test 2)
- $P(D_2 = 1 \mid H = 0) = 0.03$ (3% false positive for test 2)

**NEW:** Tests are conditionally dependent for healthy patients:
- If first test is positive for healthy patient, second test is more likely to be positive
- $P(D_2 = 1 \mid H = 0, D_1 = 1) = 0.30$ (given healthy AND first test positive)
- $P(D_2 = 1 \mid H = 0, D_1 = 0) = 0.01$ (given healthy AND first test negative)

---

### Part (a): Joint Probability Table for $D_1$ and $D_2$ Given $H = 0$

**Step 1:** Compute $P(D_1 = 1 \mid H = 0)$ and $P(D_1 = 0 \mid H = 0)$:
$$P(D_1 = 1 \mid H = 0) = 0.01$$
$$P(D_1 = 0 \mid H = 0) = 0.99$$

**Step 2:** Use conditional probabilities to compute joint:

$$P(D_1 = 1, D_2 = 1 \mid H = 0) = P(D_2 = 1 \mid H = 0, D_1 = 1) \cdot P(D_1 = 1 \mid H = 0)$$
$$= 0.30 \times 0.01 = 0.003$$

$$P(D_1 = 1, D_2 = 0 \mid H = 0) = P(D_2 = 0 \mid H = 0, D_1 = 1) \cdot P(D_1 = 1 \mid H = 0)$$
$$= 0.70 \times 0.01 = 0.007$$

$$P(D_1 = 0, D_2 = 1 \mid H = 0) = P(D_2 = 1 \mid H = 0, D_1 = 0) \cdot P(D_1 = 0 \mid H = 0)$$
$$= 0.01 \times 0.99 = 0.0099$$

$$P(D_1 = 0, D_2 = 0 \mid H = 0) = P(D_2 = 0 \mid H = 0, D_1 = 0) \cdot P(D_1 = 0 \mid H = 0)$$
$$= 0.99 \times 0.99 = 0.9801$$

**Joint Probability Table:**

|  | $D_2 = 0$ | $D_2 = 1$ | Marginal |
|---|---|---|---|
| $D_1 = 0$ | 0.9801 | 0.0099 | 0.99 |
| $D_1 = 1$ | 0.007 | 0.003 | 0.01 |
| **Marginal** | **0.9871** | **0.0129** | **1.0** |

**Verification:** $0.9801 + 0.0099 + 0.007 + 0.003 = 1.0$ ✓

---

### Part (b): Probability of Disease After One Positive Test

**Apply Bayes' Theorem:**

$$P(H = 1 \mid D_1 = 1) = \frac{P(D_1 = 1 \mid H = 1) P(H = 1)}{P(D_1 = 1)}$$

**Compute $P(D_1 = 1)$ using marginalization:**
$$P(D_1 = 1) = P(D_1 = 1 \mid H = 1) P(H = 1) + P(D_1 = 1 \mid H = 0) P(H = 0)$$
$$= (1.0)(0.0015) + (0.01)(0.9985) = 0.0015 + 0.009985 = 0.011485$$

**Apply Bayes:**
$$P(H = 1 \mid D_1 = 1) = \frac{(1.0)(0.0015)}{0.011485} = \frac{0.0015}{0.011485} \approx 0.1306$$

**Answer:** 13.06% probability of having the disease after one positive test.

---

### Part (c): Probability of Disease After Both Tests Positive

**Given:** $D_1 = 1$ and $D_2 = 1$

**Apply Bayes:**
$$P(H = 1 \mid D_1 = 1, D_2 = 1) = \frac{P(D_1 = 1, D_2 = 1 \mid H = 1) P(H = 1)}{P(D_1 = 1, D_2 = 1)}$$

**Compute numerator for $H = 1$:**

Assume tests are independent given disease:
$$P(D_1 = 1, D_2 = 1 \mid H = 1) = P(D_1 = 1 \mid H = 1) \cdot P(D_2 = 1 \mid H = 1)$$
$$= (1.0)(0.98) = 0.98$$

Therefore:
$$\text{Numerator} = 0.98 \times 0.0015 = 0.00147$$

**Compute marginal $P(D_1 = 1, D_2 = 1)$:**
$$P(D_1 = 1, D_2 = 1) = P(D_1 = 1, D_2 = 1 \mid H = 1) P(H = 1) + P(D_1 = 1, D_2 = 1 \mid H = 0) P(H = 0)$$
$$= 0.98 \times 0.0015 + 0.003 \times 0.9985$$
$$= 0.00147 + 0.0029955 = 0.0044655$$

**Apply Bayes:**
$$P(H = 1 \mid D_1 = 1, D_2 = 1) = \frac{0.00147}{0.0044655} \approx 0.3292$$

**Answer:** 32.92% probability of having the disease after both tests positive.

**Note:** This is LOWER than the independent case (83.07% from Section 8.1.2) because the conditional dependence means that when both tests are positive for a healthy person, it's more likely they're correlated false positives.

**Comparison:**

| Scenario | P(Disease \| Evidence) |
|----------|------------------------|
| One positive test | 13.06% |
| Both positive (independent) | 83.07% |
| Both positive (dependent) | 32.92% |

Conditional dependence reduces the evidential value of the second test.

In [ ]:
# Exercise 6: Markov Chain Demonstration
np.random.seed(42)

# Define a simple weather Markov chain
# States: 0 = Rainy, 1 = Sunny
# Transition matrix: P(tomorrow | today)
transition_matrix = np.array([
    [0.3, 0.7],  # If rainy today: 30% rainy, 70% sunny tomorrow
    [0.2, 0.8]   # If sunny today: 20% rainy, 80% sunny tomorrow
])

# Initial distribution
initial_dist = np.array([0.3, 0.7])  # 30% rainy, 70% sunny today

# Simulate Markov chain
n_days = 3
n_simulations = 100000

# Count occurrences of (A, B, C) combinations
counts = np.zeros((2, 2, 2))  # (day1, day2, day3)

for _ in range(n_simulations):
    # Day 1: Sample from initial distribution
    day1 = np.random.choice([0, 1], p=initial_dist)
    
    # Day 2: Sample from transition given day1
    day2 = np.random.choice([0, 1], p=transition_matrix[day1])
    
    # Day 3: Sample from transition given day2 (NOT day1!)
    day3 = np.random.choice([0, 1], p=transition_matrix[day2])
    
    counts[day1, day2, day3] += 1

# Convert counts to probabilities
empirical_joint = counts / n_simulations

# Compute theoretical joint probabilities using Markov property
theoretical_joint = np.zeros((2, 2, 2))
for a in range(2):
    for b in range(2):
        for c in range(2):
            theoretical_joint[a, b, c] = (
                initial_dist[a] * 
                transition_matrix[a, b] * 
                transition_matrix[b, c]
            )

# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

states = ['Rainy', 'Sunny']
combinations = []
empirical_probs = []
theoretical_probs = []

for a in range(2):
    for b in range(2):
        for c in range(2):
            combinations.append(f"{states[a][0]}{states[b][0]}{states[c][0]}")
            empirical_probs.append(empirical_joint[a, b, c])
            theoretical_probs.append(theoretical_joint[a, b, c])

# Plot 1: Comparison of empirical vs theoretical
ax1 = axes[0]
x = np.arange(len(combinations))
width = 0.35
ax1.bar(x - width/2, empirical_probs, width, label='Empirical', alpha=0.8)
ax1.bar(x + width/2, theoretical_probs, width, label='Theoretical (Markov)', alpha=0.8)
ax1.set_xlabel('Weather sequence (Day1-Day2-Day3)')
ax1.set_ylabel('Probability')
ax1.set_title('Joint Distribution P(A, B, C)')
ax1.set_xticks(x)
ax1.set_xticklabels(combinations, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Markov chain diagram
ax2 = axes[1]
ax2.text(0.1, 0.5, 'A', fontsize=20, ha='center', 
         bbox=dict(boxstyle='circle', facecolor='lightblue', edgecolor='black', linewidth=2))
ax2.text(0.5, 0.5, 'B', fontsize=20, ha='center',
         bbox=dict(boxstyle='circle', facecolor='lightgreen', edgecolor='black', linewidth=2))
ax2.text(0.9, 0.5, 'C', fontsize=20, ha='center',
         bbox=dict(boxstyle='circle', facecolor='lightcoral', edgecolor='black', linewidth=2))
ax2.arrow(0.15, 0.5, 0.3, 0, head_width=0.05, head_length=0.05, fc='black', ec='black')
ax2.arrow(0.55, 0.5, 0.3, 0, head_width=0.05, head_length=0.05, fc='black', ec='black')
ax2.text(0.3, 0.6, 'P(B|A)', fontsize=12, ha='center')
ax2.text(0.7, 0.6, 'P(C|B)', fontsize=12, ha='center')
ax2.set_xlim([0, 1])
ax2.set_ylim([0.3, 0.7])
ax2.axis('off')
ax2.set_title('Markov Chain Structure')

# Plot 3: Error analysis
ax3 = axes[2]
errors = np.abs(np.array(empirical_probs) - np.array(theoretical_probs))
ax3.bar(range(len(combinations)), errors, color='orange', alpha=0.7)
ax3.set_xlabel('Weather sequence')
ax3.set_ylabel('|Empirical - Theoretical|')
ax3.set_title('Approximation Error')
ax3.set_xticks(range(len(combinations)))
ax3.set_xticklabels(combinations, rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print detailed results
print("="*70)
print("EXERCISE 6 RESULTS: MARKOV CHAIN")
print("="*70)
print(f"\nSimulations: {n_simulations:,}\n")
print("Transition Matrix P(Tomorrow | Today):")
print("                   Rainy    Sunny")
print(f"  If Rainy today:  {transition_matrix[0, 0]:.2f}     {transition_matrix[0, 1]:.2f}")
print(f"  If Sunny today:  {transition_matrix[1, 0]:.2f}     {transition_matrix[1, 1]:.2f}")
print(f"\nInitial Distribution P(Day 1):")
print(f"  Rainy: {initial_dist[0]:.2f}, Sunny: {initial_dist[1]:.2f}")

print("\n" + "-"*70)
print("Joint Probability P(A, B, C) = P(A) · P(B|A) · P(C|B)")
print("-"*70)
print(f"{'Sequence':<10} {'Empirical':<12} {'Theoretical':<12} {'Error':<10}")
print("-"*70)

for i, combo in enumerate(combinations):
    print(f"{combo:<10} {empirical_probs[i]:11.6f}  {theoretical_probs[i]:11.6f}  {errors[i]:9.6f}")

print("\nMax error:", max(errors))
print("Mean error:", np.mean(errors))
print("\nConclusion: Markov factorization P(A)·P(B|A)·P(C|B) accurately models the joint distribution")

---

## Exercise 6: Markov Chain Simplification

> "Assume that we have a sequence of random variables A, B, and C, where B only depends on A, and C only depends on B, can you simplify the joint probability P(A, B, C)? Hint: this is a Markov chain."

### Solution

**Given Structure:**
- $B$ only depends on $A$: $B \perp \!\!\! \perp \text{(past)} \mid A$
- $C$ only depends on $B$: $C \perp \!\!\! \perp A \mid B$

This is a **Markov chain**: $A \to B \to C$

---

**Derivation:**

**Step 1:** Start with the chain rule of probability:
$$P(A, B, C) = P(C \mid A, B) \cdot P(B \mid A) \cdot P(A)$$

**Step 2:** Apply the Markov property: $C$ only depends on $B$, so given $B$, $C$ is independent of $A$:
$$P(C \mid A, B) = P(C \mid B)$$

*Justification:* Conditional independence $C \perp \!\!\! \perp A \mid B$.

**Step 3:** Substitute into Step 1:
$$P(A, B, C) = P(C \mid B) \cdot P(B \mid A) \cdot P(A)$$

**Final Answer:**
$$\boxed{P(A, B, C) = P(A) \cdot P(B \mid A) \cdot P(C \mid B)}$$

---

**Interpretation:**
- Start at $A$ with probability $P(A)$
- Transition to $B$ with probability $P(B \mid A)$
- Transition to $C$ with probability $P(C \mid B)$

The joint probability is the product of these transition probabilities.

---

**General Markov Chain:**

For a Markov chain $X_1 \to X_2 \to \cdots \to X_n$:
$$P(X_1, X_2, \ldots, X_n) = P(X_1) \prod_{i=2}^n P(X_i \mid X_{i-1})$$

This is a massive simplification:
- **Without Markov property:** $O(k^n)$ parameters (full joint distribution)
- **With Markov property:** $O(nk^2)$ parameters (transition probabilities)

---

**Verification:**

Expand $P(C \mid B) \cdot P(B \mid A) \cdot P(A)$:
$$= \frac{P(B, C)}{P(B)} \cdot \frac{P(A, B)}{P(A)} \cdot P(A) = \frac{P(B, C) \cdot P(A, B)}{P(B)}$$

Using $P(A, B, C) = P(C \mid A, B) P(A, B)$ and $P(C \mid A, B) = P(C \mid B)$:
$$= \frac{P(B, C) \cdot P(A, B)}{P(B)} = P(A, B, C) \quad \checkmark$$

---

**Example Application:**

**Weather Model:**
- $A$: Weather today (Sunny/Rainy)
- $B$: Weather tomorrow (depends only on today)
- $C$: Weather day after tomorrow (depends only on tomorrow)

If today is sunny ($P(A = \text{Sunny}) = 0.7$), tomorrow has 80% chance of being sunny given today is sunny ($P(B = \text{Sunny} \mid A = \text{Sunny}) = 0.8$), and the day after tomorrow has 75% chance of being sunny given tomorrow is sunny ($P(C = \text{Sunny} \mid B = \text{Sunny}) = 0.75$):

$$P(\text{All Sunny}) = 0.7 \times 0.8 \times 0.75 = 0.42$$

Without the Markov property, we'd need to know $P(C = \text{Sunny} \mid A = \text{Sunny}, B = \text{Sunny})$, which depends on two days of history.

In [ ]:
# Exercise 5: Visualization of probability bounds
import matplotlib.patches as mpatches
from matplotlib.patches import Circle, Rectangle

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Test cases
test_cases = [
    (0.3, 0.3, "P(A)=0.3, P(B)=0.3"),
    (0.6, 0.4, "P(A)=0.6, P(B)=0.4"),
    (0.8, 0.7, "P(A)=0.8, P(B)=0.7")
]

for idx, (p_a, p_b, title) in enumerate(test_cases):
    # Compute bounds
    union_lower = max(p_a, p_b)
    union_upper = min(1.0, p_a + p_b)
    
    intersect_lower = max(0, p_a + p_b - 1)
    intersect_upper = min(p_a, p_b)
    
    # Plot union bounds
    ax1 = axes[0, idx]
    scenarios_union = [union_lower, (union_lower + union_upper)/2, union_upper]
    labels_union = ['Min\n(one contains other)', 'Middle', 'Max\n(disjoint)']
    colors_union = ['lightblue', 'lightyellow', 'lightcoral']
    
    ax1.bar(range(3), scenarios_union, color=colors_union, edgecolor='black', linewidth=2)
    ax1.axhline(y=union_lower, color='green', linestyle='--', linewidth=2, label=f'Lower: {union_lower:.2f}')
    ax1.axhline(y=union_upper, color='red', linestyle='--', linewidth=2, label=f'Upper: {union_upper:.2f}')
    ax1.set_ylabel('P(A ∪ B)')
    ax1.set_title(f'Union: {title}')
    ax1.set_xticks(range(3))
    ax1.set_xticklabels(labels_union, fontsize=8)
    ax1.set_ylim([0, 1.1])
    ax1.legend(fontsize=8)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Plot intersection bounds
    ax2 = axes[1, idx]
    scenarios_intersect = [intersect_lower, (intersect_lower + intersect_upper)/2, intersect_upper]
    labels_intersect = ['Min\n(disjoint)', 'Middle', 'Max\n(one contains other)']
    colors_intersect = ['lightcoral', 'lightyellow', 'lightblue']
    
    ax2.bar(range(3), scenarios_intersect, color=colors_intersect, edgecolor='black', linewidth=2)
    ax2.axhline(y=intersect_lower, color='green', linestyle='--', linewidth=2, label=f'Lower: {intersect_lower:.2f}')
    ax2.axhline(y=intersect_upper, color='red', linestyle='--', linewidth=2, label=f'Upper: {intersect_upper:.2f}')
    ax2.set_ylabel('P(A ∩ B)')
    ax2.set_title(f'Intersection: {title}')
    ax2.set_xticks(range(3))
    ax2.set_xticklabels(labels_intersect, fontsize=8)
    ax2.set_ylim([0, max(1.0, intersect_upper * 1.2)])
    ax2.legend(fontsize=8)
    ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print detailed results
print("="*70)
print("EXERCISE 5 RESULTS: PROBABILITY BOUNDS")
print("="*70)

for p_a, p_b, desc in test_cases:
    print(f"\n{desc}")
    print("-" * 70)
    
    union_lower = max(p_a, p_b)
    union_upper = min(1.0, p_a + p_b)
    intersect_lower = max(0, p_a + p_b - 1)
    intersect_upper = min(p_a, p_b)
    
    print(f"\nP(A ∪ B) bounds:")
    print(f"  Lower bound: max{{P(A), P(B)}} = max{{{p_a}, {p_b}}} = {union_lower}")
    print(f"  Upper bound: min{{1, P(A)+P(B)}} = min{{1, {p_a+p_b}}} = {union_upper}")
    print(f"  Range: [{union_lower:.3f}, {union_upper:.3f}]")
    
    print(f"\nP(A ∩ B) bounds:")
    print(f"  Lower bound: max{{0, P(A)+P(B)-1}} = max{{0, {p_a+p_b-1:.1f}}} = {intersect_lower}")
    print(f"  Upper bound: min{{P(A), P(B)}} = min{{{p_a}, {p_b}}} = {intersect_upper}")
    print(f"  Range: [{intersect_lower:.3f}, {intersect_upper:.3f}]")
    
    # Verify with extreme cases
    print(f"\nVerification:")
    print(f"  If disjoint: P(A∪B) = {p_a+p_b:.2f}, P(A∩B) = 0")
    print(f"  If A⊆B: P(A∪B) = {p_b:.2f}, P(A∩B) = {min(p_a, p_b):.2f}")
    
print("\n" + "="*70)

---

## Exercise 5: Probability Bounds for Union and Intersection

> "Given two events with probability P(A) and P(B), compute upper and lower bounds on P(A ∪ B) and P(A ∩ B). Hint: graph the situation using a Venn diagram."

### Solution

**Given:**
- $P(A)$ and $P(B)$ are known
- Need bounds on $P(A \cup B)$ and $P(A \cap B)$

---

### Part (a): Bounds on $P(A \cup B)$

**Inclusion-Exclusion Principle:**
$$P(A \cup B) = P(A) + P(B) - P(A \cap B)$$

**Upper Bound:**

Since $P(A \cap B) \geq 0$:
$$P(A \cup B) = P(A) + P(B) - P(A \cap B) \leq P(A) + P(B)$$

**Upper bound:** $P(A \cup B) \leq P(A) + P(B)$

The bound is tight when $A$ and $B$ are disjoint: $P(A \cap B) = 0$.

**Lower Bound:**

Since $A \cup B$ contains both $A$ and $B$:
$$P(A \cup B) \geq \max\{P(A), P(B)\}$$

Also, using $P(A \cap B) \leq \min\{P(A), P(B)\}$ in inclusion-exclusion:
$$P(A \cup B) \geq P(A) + P(B) - \min\{P(A), P(B)\} = \max\{P(A), P(B)\}$$

**Lower bound:** $P(A \cup B) \geq \max\{P(A), P(B)\}$

The bound is tight when $A \subseteq B$ or $B \subseteq A$.

**Summary for Union:**
$$\boxed{\max\{P(A), P(B)\} \leq P(A \cup B) \leq \min\{1, P(A) + P(B)\}}$$

Note: We add $\min\{1, \cdot\}$ since probabilities cannot exceed 1.

---

### Part (b): Bounds on $P(A \cap B)$

**Upper Bound:**

Since $A \cap B \subseteq A$ and $A \cap B \subseteq B$:
$$P(A \cap B) \leq \min\{P(A), P(B)\}$$

**Upper bound:** $P(A \cap B) \leq \min\{P(A), P(B)\}$

The bound is tight when $A \subseteq B$ or $B \subseteq A$.

**Lower Bound:**

From inclusion-exclusion:
$$P(A \cap B) = P(A) + P(B) - P(A \cup B)$$

Since $P(A \cup B) \leq 1$:
$$P(A \cap B) \geq P(A) + P(B) - 1$$

Also, probabilities are non-negative:
$$P(A \cap B) \geq 0$$

**Lower bound:** $P(A \cap B) \geq \max\{0, P(A) + P(B) - 1\}$

The bound $P(A) + P(B) - 1$ is called the **Fréchet-Hoeffding lower bound**.

**Summary for Intersection:**
$$\boxed{\max\{0, P(A) + P(B) - 1\} \leq P(A \cap B) \leq \min\{P(A), P(B)\}}$$

---

### Examples

**Example 1:** $P(A) = 0.6$, $P(B) = 0.4$
- Union: $\max\{0.6, 0.4\} = 0.6 \leq P(A \cup B) \leq 1.0$
- Intersection: $\max\{0, 0.6+0.4-1\} = 0 \leq P(A \cap B) \leq \min\{0.6, 0.4\} = 0.4$

**Example 2:** $P(A) = 0.8$, $P(B) = 0.7$
- Union: $\max\{0.8, 0.7\} = 0.8 \leq P(A \cup B) \leq 1.0$
- Intersection: $\max\{0, 0.8+0.7-1\} = 0.5 \leq P(A \cap B) \leq \min\{0.8, 0.7\} = 0.7$

---

### Graphical Interpretation (Venn Diagram)

```
        ┌─────────────┐
        │      A      │
        │   ┌─────────┼─────┐
        │   │ A ∩ B   │     │
        │   │         │  B  │
        └───┼─────────┘     │
            └───────────────┘
```

- **Maximum overlap:** $A \cap B = \min\{A, B\}$ (one set contains the other)
- **Minimum overlap:** $A \cap B = \max\{0, P(A) + P(B) - 1\}$
- **Maximum union:** $A \cup B$ approaches full space when sets are disjoint
- **Minimum union:** $A \cup B = \max\{A, B\}$ when one set contains the other

In [ ]:
# Exercise 4: Demonstration of dependence between overlapping averages
np.random.seed(42)

# Generate data
n_samples = 1000
data = np.random.normal(0, 1, n_samples)

# Compute multiple overlapping averages
window_size = 100
n_windows = n_samples - window_size + 1

averages = []
for i in range(n_windows):
    window_avg = np.mean(data[i:i+window_size])
    averages.append(window_avg)

averages = np.array(averages)

# Compute correlation between consecutive averages
correlations = []
for lag in range(1, min(200, n_windows)):
    corr = np.corrcoef(averages[:-lag], averages[lag:])[0, 1]
    correlations.append(corr)

plt.figure(figsize=(14, 5))

# Plot the averages
plt.subplot(1, 3, 1)
plt.plot(averages, alpha=0.7, linewidth=0.8)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
epsilon = 0.1
plt.axhline(y=epsilon, color='orange', linestyle='--', alpha=0.5, label=f'±ε = ±{epsilon}')
plt.axhline(y=-epsilon, color='orange', linestyle='--', alpha=0.5)
plt.xlabel('Window index')
plt.ylabel('Average value')
plt.title(f'Overlapping Averages (window size={window_size})')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot correlation vs lag
plt.subplot(1, 3, 2)
plt.plot(range(1, len(correlations)+1), correlations, linewidth=2)
plt.xlabel('Lag between averages')
plt.ylabel('Correlation coefficient')
plt.title('Correlation Between Overlapping Averages')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)

# Compare independent vs dependent case
plt.subplot(1, 3, 3)
n_trials = 10000
m = 100
epsilon = 0.2

# Single average (correct application)
single_violations = 0
for _ in range(n_trials):
    sample = np.random.normal(0, 1, m)
    avg = np.mean(sample)
    if abs(avg) >= epsilon:
        single_violations += 1

# Multiple overlapping averages (problematic)
k_averages = 10
multiple_violations = 0
for _ in range(n_trials):
    sample = np.random.normal(0, 1, m + k_averages - 1)
    avgs = [np.mean(sample[i:i+m]) for i in range(k_averages)]
    if any(abs(a) >= epsilon for a in avgs):
        multiple_violations += 1

theoretical_single = 1 / (m * epsilon**2)
naive_multiple = min(1.0, k_averages / (m * epsilon**2))  # Wrong!

categories = ['Single\nAverage', 'Multiple\nOverlapping\nAverages']
empirical_probs = [single_violations/n_trials, multiple_violations/n_trials]
theoretical_probs = [theoretical_single, naive_multiple]

x = np.arange(len(categories))
width = 0.35

plt.bar(x - width/2, empirical_probs, width, label='Empirical', alpha=0.8)
plt.bar(x + width/2, theoretical_probs, width, label='Chebyshev (naive)', alpha=0.8)

plt.ylabel('P(|average| ≥ ε)')
plt.title(f'Chebyshev Application (ε={epsilon}, m={m})')
plt.xticks(x, categories)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("="*70)
print("EXERCISE 4 RESULTS")
print("="*70)
print(f"\nSetup: m = {m} samples, ε = {epsilon}, {n_trials:,} trials\n")
print("Single Average:")
print(f"  Empirical P(|avg| ≥ ε): {single_violations/n_trials:.4f}")
print(f"  Chebyshev bound: ≤ {theoretical_single:.4f}")
print(f"  Ratio: {(single_violations/n_trials)/theoretical_single:.2f} (Chebyshev is conservative)\n")

print(f"Multiple Overlapping Averages (k={k_averages}):")
print(f"  Empirical P(any |avg| ≥ ε): {multiple_violations/n_trials:.4f}")
print(f"  Naive bound (treating as independent): ≤ {naive_multiple:.4f}")
print(f"  Actual violation rate is higher due to correlation!\n")

print(f"Correlation Analysis:")
print(f"  Correlation(avg_i, avg_{{i+1}}): {correlations[0]:.4f}")
print(f"  Correlation(avg_i, avg_{{i+10}}): {correlations[9]:.4f}")
print(f"  High correlation confirms dependence between overlapping averages")

---

## Exercise 4: Independence of Sample Averages

> "Assume that we draw m samples from a probability distribution with zero mean and unit variance. Compute the averages. Can we apply Chebyshev's inequality for every average independently? Why not?"

### Solution

**Setup:**
- Draw $m$ samples: $X_1, X_2, \ldots, X_m$ i.i.d. from distribution with $E[X_i] = 0$ and $\text{Var}[X_i] = 1$
- Compute sample average: $\bar{X}_m = \frac{1}{m}\sum_{i=1}^m X_i$

**Properties of the Average:**

$$E[\bar{X}_m] = \frac{1}{m}\sum_{i=1}^m E[X_i] = 0$$

$$\text{Var}[\bar{X}_m] = \frac{1}{m^2}\sum_{i=1}^m \text{Var}[X_i] = \frac{m}{m^2} = \frac{1}{m}$$

**Applying Chebyshev to a Single Average:**

For one sample average, Chebyshev's inequality states:
$$P(|\bar{X}_m| \geq \epsilon) \leq \frac{1}{m\epsilon^2}$$

This is valid for a **single** average.

---

**Can We Apply This to Multiple Averages Independently?**

**Answer: NO - We cannot apply Chebyshev independently to multiple averages from the same data.**

**Reasoning:**

1. **Dependence Through Shared Data:**
   - If we compute multiple overlapping averages (e.g., $\bar{X}_{m_1}$, $\bar{X}_{m_2}$ using subsets of the same data), they are **not independent**
   - Shared samples create correlation between the averages

2. **Union Bound Issue:**
   - If we want $P(\text{all k averages within } \epsilon) \geq 1-\delta$, we might naively use:
   $$P(\text{at least one average exceeds } \epsilon) \leq \sum_{j=1}^k P(|\bar{X}_j| \geq \epsilon)$$
   - This only works if the events are **mutually exclusive or positively correlated**, not generally dependent

3. **Correct Approach:**
   - Use **union bound** (Bonferroni correction): Require each average to satisfy Chebyshev with probability $1 - \delta/k$
   - Or use **simultaneous concentration inequalities** (e.g., maximal inequalities)

**Example:**

Suppose we compute $k$ different averages from overlapping data:
- $\bar{X}_1 = \frac{1}{m}\sum_{i=1}^m X_i$
- $\bar{X}_2 = \frac{1}{m}\sum_{i=2}^{m+1} X_i$
- ...

These averages share $m-1$ samples, making them **highly correlated**.

**Correct Statement:**

For **independent** samples (no overlap), we can apply Chebyshev to each independently. But if samples overlap or if we're making multiple statements about the same data, we need to account for dependence through:
- Union bounds
- Bonferroni correction  
- Simultaneous inference methods

**Key Lesson:** Independence of random variables does not automatically grant independence of statements about them when applied to the same realization of data.

In [ ]:
# Exercise 3: Variance scaling and Chebyshev bounds
from scipy import stats

p = 0.5  # Fair coin
epsilon = 0.01  # Desired accuracy
confidence = 0.95

# Part (a): Variance scaling
n_values = np.array([10, 50, 100, 500, 1000, 5000, 10000])
variances = p * (1 - p) / n_values

plt.figure(figsize=(12, 9))

# Variance scaling
plt.subplot(2, 2, 1)
plt.plot(n_values, variances, 'o-', linewidth=2, markersize=8)
plt.xlabel('Number of samples (n)')
plt.ylabel('Variance of estimate')
plt.title('Part (a): Variance scales as 1/n')
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.yscale('log')

# Part (b): Chebyshev bound
plt.subplot(2, 2, 2)
chebyshev_bound = 1 / (4 * n_values * epsilon**2)
plt.plot(n_values, chebyshev_bound, 'o-', linewidth=2, markersize=8, label='Chebyshev bound')
plt.axhline(y=1-confidence, color='r', linestyle='--', label=f'Target: {1-confidence}')
plt.xlabel('Number of samples (n)')
plt.ylabel('P(|estimate - p| ≥ ε)')
plt.title(f'Part (b): Chebyshev Bound (ε={epsilon})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.yscale('log')

# Part (c): CLT comparison
plt.subplot(2, 2, 3)
n_range = np.arange(1000, 60000, 500)
chebyshev_prob = 1 / (4 * n_range * epsilon**2)
clt_prob = 2 * (1 - stats.norm.cdf(epsilon * np.sqrt(n_range) / 0.5))

plt.plot(n_range, chebyshev_prob, label='Chebyshev bound', linewidth=2)
plt.plot(n_range, clt_prob, label='CLT (Normal approximation)', linewidth=2)
plt.axhline(y=1-confidence, color='r', linestyle='--', label=f'Target: {1-confidence}')
plt.xlabel('Number of samples (n)')
plt.ylabel('P(|estimate - p| ≥ ε)')
plt.title('Part (c): Chebyshev vs CLT')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Empirical verification
plt.subplot(2, 2, 4)
np.random.seed(42)
n_test = 1000
n_simulations = 10000

errors = []
for _ in range(n_simulations):
    sample = np.random.binomial(1, p, n_test)
    p_hat = np.mean(sample)
    errors.append(abs(p_hat - p))

empirical_prob = np.mean(np.array(errors) >= epsilon)
theoretical_std = np.sqrt(p * (1 - p) / n_test)

plt.hist(errors, bins=50, density=True, alpha=0.7, label=f'Empirical (n={n_test})')
x = np.linspace(0, max(errors), 1000)
# PDF of |Z| where Z ~ N(0, σ²) is a folded normal
plt.axvline(x=epsilon, color='r', linestyle='--', linewidth=2, label=f'ε = {epsilon}')
plt.xlabel('|estimate - p|')
plt.ylabel('Density')
plt.title(f'Empirical Distribution of Estimation Error')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Numerical results
print("="*70)
print("EXERCISE 3 RESULTS")
print("="*70)
print(f"\nPart (a): Variance Scaling")
print(f"For p = {p}:")
for n in [100, 1000, 10000]:
    var = p * (1 - p) / n
    print(f"  n = {n:5d}: Var[p̂_n] = {var:.6f} = 1/{int(1/var)}")

print(f"\nPart (b): Chebyshev Bound (ε = {epsilon}, confidence = {confidence})")
# Calculate required n for Chebyshev
n_chebyshev = int(np.ceil(1 / (4 * (1 - confidence) * epsilon**2)))
print(f"  Required samples (Chebyshev): n ≥ {n_chebyshev:,}")

print(f"\nPart (c): CLT Comparison")
# Calculate required n for CLT
z_alpha = stats.norm.ppf(1 - (1 - confidence) / 2)
n_clt = int(np.ceil((z_alpha * 0.5 / epsilon)**2))
print(f"  Required samples (CLT): n ≥ {n_clt:,}")
print(f"  Improvement: {n_chebyshev / n_clt:.1f}x fewer samples needed with CLT")

print(f"\nEmpirical Verification (n = {n_test}, {n_simulations:,} simulations):")
print(f"  P(|p̂ - p| ≥ {epsilon}) ≈ {empirical_prob:.4f}")
print(f"  Chebyshev bound: ≤ {1/(4*n_test*epsilon**2):.4f}")
print(f"  CLT prediction: ≈ {2 * (1 - stats.norm.cdf(epsilon * np.sqrt(n_test) / 0.5)):.4f}")

---

## Exercise 3: Variance Analysis for Coin Tosses

> "We empirically demonstrated convergence to the mean for the toss of a coin. Calculate the variance of the estimate of the probability that we see a head after drawing n samples."

### Part (a): Variance Scaling with Observations

**Setup:**
- Let $X_1, X_2, \ldots, X_n$ be i.i.d. Bernoulli$(p)$ random variables (coin flips)
- Estimator: $\hat{p}_n = \frac{1}{n}\sum_{i=1}^n X_i$

**Derivation:**

$$\text{Var}[\hat{p}_n] = \text{Var}\left[\frac{1}{n}\sum_{i=1}^n X_i\right]$$

By linearity of variance for independent random variables:
$$= \frac{1}{n^2}\sum_{i=1}^n \text{Var}[X_i]$$

Since each $X_i \sim \text{Bernoulli}(p)$ with $\text{Var}[X_i] = p(1-p)$:
$$= \frac{1}{n^2} \cdot n \cdot p(1-p) = \frac{p(1-p)}{n}$$

**Answer:** The variance scales as $O(1/n)$.

**For a fair coin** ($p = 0.5$):
$$\text{Var}[\hat{p}_n] = \frac{0.5 \times 0.5}{n} = \frac{0.25}{n} = \frac{1}{4n}$$

---

### Part (b): Chebyshev Bound

**Chebyshev's Inequality:**
For any $\epsilon > 0$:
$$P(|\hat{p}_n - p| \geq \epsilon) \leq \frac{\text{Var}[\hat{p}_n]}{\epsilon^2} = \frac{p(1-p)}{n\epsilon^2}$$

**For a fair coin** ($p = 0.5$):
$$P(|\hat{p}_n - 0.5| \geq \epsilon) \leq \frac{1}{4n\epsilon^2}$$

**Example:** To ensure the estimate is within $\epsilon = 0.01$ of the true value with probability at least 0.95:
$$P(|\hat{p}_n - p| < 0.01) \geq 0.95$$

This requires:
$$\frac{1}{4n(0.01)^2} \leq 0.05$$

Solving for $n$:
$$n \geq \frac{1}{4 \times 0.05 \times 0.0001} = \frac{1}{0.00002} = 50{,}000$$

**Chebyshev gives a conservative bound** - in practice, fewer samples suffice.

---

### Part (c): Relation to Central Limit Theorem

**Central Limit Theorem Statement:**
$$\frac{\hat{p}_n - p}{\sqrt{p(1-p)/n}} \xrightarrow{d} \mathcal{N}(0, 1)$$

Equivalently:
$$\hat{p}_n \approx \mathcal{N}\left(p, \frac{p(1-p)}{n}\right) \quad \text{for large } n$$

**Connection:**
1. **Variance:** Both approaches give $\text{Var}[\hat{p}_n] = \frac{p(1-p)}{n}$
2. **Convergence rate:** Both show $O(1/\sqrt{n})$ standard error
3. **Distribution shape:** CLT provides additional information - the distribution is approximately Gaussian

**Improved Bounds with CLT:**
For $p = 0.5$ and large $n$:
$$P(|\hat{p}_n - 0.5| \geq \epsilon) \approx 2\Phi\left(-\frac{\epsilon\sqrt{n}}{0.5}\right)$$

where $\Phi$ is the standard normal CDF.

For $\epsilon = 0.01$ and 95% confidence:
$$2\Phi\left(-\frac{0.01\sqrt{n}}{0.5}\right) \leq 0.05$$

This gives $n \approx 9{,}604$ (much better than Chebyshev's $50{,}000$).

**Summary:**

| Approach | Sample size for 95% confidence, $\epsilon = 0.01$ |
|----------|---------------------------------------------------|
| Chebyshev | $n \geq 50{,}000$ |
| CLT (Normal approximation) | $n \approx 9{,}604$ |

The CLT provides tighter bounds by using distributional information beyond just variance.

In [ ]:
# Exercise 2: Demonstration of aleatoric uncertainty limit
np.random.seed(42)

p_true = 0.7

# Even with perfect knowledge of p, individual outcomes remain uncertain
n_trials = 1000
outcomes = np.random.binomial(1, p_true, n_trials)

# Compute running statistics
running_mean = np.cumsum(outcomes) / np.arange(1, n_trials + 1)
running_var = []
for i in range(1, n_trials + 1):
    running_var.append(np.var(outcomes[:i]))

theoretical_var = p_true * (1 - p_true)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(running_mean, alpha=0.7, label='Estimated mean')
plt.axhline(y=p_true, color='r', linestyle='--', label=f'True mean: {p_true}')
plt.xlabel('Number of observations')
plt.ylabel('Estimated probability')
plt.title('Epistemic Uncertainty: Mean Estimate Converges')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(running_var, alpha=0.7, label='Estimated variance')
plt.axhline(y=theoretical_var, color='r', linestyle='--', 
            label=f'True variance: {theoretical_var:.3f}')
plt.xlabel('Number of observations')
plt.ylabel('Estimated variance')
plt.title('Aleatoric Uncertainty: Variance Remains Constant')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"True probability: {p_true}")
print(f"True variance (aleatoric): {theoretical_var:.4f}")
print(f"\nEstimated mean converges to: {running_mean[-1]:.4f}")
print(f"Estimated variance stabilizes at: {running_var[-1]:.4f}")
print(f"\nEpistemic uncertainty (about mean): Can be reduced to ~0")
print(f"Aleatoric uncertainty (variance): Remains at {theoretical_var:.4f}")

---

## Exercise 2: Aleatoric Uncertainty Limit

> "Give an example where observing more data will only reduce the amount of uncertainty up to a point and then no further. Explain why this is the case and where you expect this point to occur."

### Solution

**Example: Predicting Individual Coin Flips**

Consider predicting the outcome of a single coin flip from a coin with known bias $p = 0.7$.

**Setup:**
- Known parameter: $p = 0.7$ (probability of heads)
- Task: Predict the outcome of the next flip
- Uncertainty type: Aleatoric (irreducible randomness)

**Analysis:**

Even with infinite historical data, we know:
$$P(X = \text{Heads}) = 0.7$$
$$P(X = \text{Tails}) = 0.3$$

The entropy (measure of uncertainty) is:
$$H(X) = -p\log_2(p) - (1-p)\log_2(1-p) = -0.7\log_2(0.7) - 0.3\log_2(0.3) \approx 0.881 \text{ bits}$$

This entropy represents the **irreducible uncertainty** in the outcome.

**Variance:**
$$\text{Var}[X] = p(1-p) = 0.7 \times 0.3 = 0.21$$

This variance cannot be reduced by collecting more data - it is an intrinsic property of the random process.

**The Stopping Point:**
- **Epistemic uncertainty** reduces to zero once we accurately estimate $p$
- **Aleatoric uncertainty** remains at $\text{Var}[X] = 0.21$ regardless of sample size
- The stopping point occurs when our estimate $\hat{p}_n$ converges to the true $p$

**Key Distinction:**

| Type | Can be reduced? | Example |
|------|----------------|---------|
| **Epistemic** | Yes, with more data | Uncertainty about $p$ |
| **Aleatoric** | No, intrinsic randomness | Uncertainty about next flip given $p$ |

**Other Examples:**
- Quantum measurement outcomes
- Thermal noise in physical systems
- Individual customer purchase decisions (even with perfect demographic knowledge)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Exercise 1: Demonstration of epistemic uncertainty reduction
np.random.seed(42)

# True parameter (unknown in practice)
p_true = 0.6

# Simulate coin flips and track running estimate
n_max = 10000
flips = np.random.binomial(1, p_true, n_max)
running_estimate = np.cumsum(flips) / np.arange(1, n_max + 1)

# Plot convergence
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(running_estimate, alpha=0.7, linewidth=0.8)
plt.axhline(y=p_true, color='r', linestyle='--', label=f'True value: {p_true}')
plt.xlabel('Number of observations (n)')
plt.ylabel('Estimated probability')
plt.title('Convergence of Parameter Estimate')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Compute theoretical variance bound
n_values = np.arange(1, n_max + 1)
variance_bound = 1 / (4 * n_values)
plt.plot(n_values, variance_bound, label='Theoretical variance bound: 1/(4n)')
plt.xlabel('Number of observations (n)')
plt.ylabel('Variance upper bound')
plt.title('Epistemic Uncertainty Decreases with Data')
plt.legend()
plt.yscale('log')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"True probability: {p_true}")
print(f"Estimate after 100 flips: {running_estimate[99]:.4f}")
print(f"Estimate after 1000 flips: {running_estimate[999]:.4f}")
print(f"Estimate after 10000 flips: {running_estimate[9999]:.4f}")
print(f"\nVariance bound decreases from {1/(4*100):.6f} (n=100) to {1/(4*10000):.6f} (n=10000)")

---

# Exercises

## Exercise 1: Epistemic Uncertainty Reduction

> "Give an example where observing more data can reduce the amount of uncertainty about the outcome to an arbitrarily low level."

### Solution

**Example: Estimating a Fair Coin's Bias**

Consider estimating the probability $p$ that a coin lands heads. This is epistemic uncertainty - uncertainty about the parameter $p$.

**Setup:**
- Unknown parameter: $p$ (true probability of heads)
- Observable data: $X_1, X_2, \ldots, X_n$ i.i.d. Bernoulli$(p)$
- Estimator: $\hat{p}_n = \frac{1}{n}\sum_{i=1}^n X_i$

**Analysis:**

By the Law of Large Numbers:
$$\hat{p}_n \xrightarrow{P} p \quad \text{as} \quad n \to \infty$$

The variance of our estimator is:
$$\text{Var}[\hat{p}_n] = \frac{p(1-p)}{n} \leq \frac{1}{4n}$$

This decreases as $O(1/n)$, approaching zero as $n \to \infty$.

By Chebyshev's inequality, for any $\epsilon > 0$:
$$P(|\hat{p}_n - p| \geq \epsilon) \leq \frac{p(1-p)}{n\epsilon^2} \leq \frac{1}{4n\epsilon^2}$$

**Conclusion:** As $n \to \infty$, the probability that our estimate deviates from the true value by more than $\epsilon$ approaches zero. We can make uncertainty arbitrarily small by collecting more data.

**Other Examples:**
- Estimating population mean from samples
- Estimating parameters of a known distribution family
- Measuring a physical constant with repeated experiments